# definitely not production code

hashtags: #edit #kill #todo #note #float32 #new #del 1/0

debug vars: SEE DEBUG TEST PROD BUG

In [1]:
# from common import *
# from pandas.api.types import CategoricalDtype
# import warnings 
# warnings.filterwarnings('ignore')
# %matplotlib inline
# canvas(12,12)

In [2]:
# ho_time = pd.Timestamp('2015-07-01',tz='UTC')

# def fake(*,M,N):
#     M_, N_ = M, N
#     for time, M in M_.groupby('time'):
#         if time<ho_time:
#             continue
#         M['quarter'] = M.time.dt.year+(M.time.dt.quarter-1)/4
#         P = M[excluded_columns]
#         M = M.drop(columns=['returnsOpenNextMktres10','quarter'])
#         P['confidenceValue'] = 0.
#         yield M, None, P

In [3]:
# from common import *

### important devops code

In [149]:
TEST = False
SEE = lambda*a,**k:None
TRAIN = True

# #################################################### START HERE

In [5]:
import pandas as pd
M = pd.read_pickle('../data/given/M.pkl') # market obs
N = None #pd.read_pickle('../data/given/N.pkl') # news
#test = pd.read_pickle('../data/given/test.pkl')

In [6]:
def _make_env():
    '''this creates a testing version of the actual env from Kaggle kernels'''
    class __env__():
        _submitted = []
        # -1 = begin, 0 = just yielded, 1 = just predicted, -2 = finished, -3 = finished and submission file written too
        _states = [-1]

        class KaggleEnvError(Exception):
            pass

        @staticmethod
        def get_training_data():
            global N
            try:
                return M, N
            except NameError:
                return M, None

        @staticmethod
        def get_prediction_days():
            env = __class__

            class iter_get_prediction_days():
                def __iter__(self):
                    if env._states[-1] != -1:
                        raise env.KaggleEnvError('can only call get_prediction_days once!')
                    
                    global test
                    test = pd.read_pickle('../data/given/test.pkl')
                    env._days_iter = test.__iter__()
                    return self

                def __next__(self):
                    if env._states[-1] == 0:
                        raise env.KaggleEnvError('can only yield next day after the previous day was submitted')
                    elif env._states[-1] not in [-1, 1, -2, -3]:
                        raise env.KaggleEnvError(
                            'environment internal error [bad state {} for __next__]'.format(env._states[-1]))

                    try:
                        ret = env._days_iter.__next__()
                    except StopIteration:
                        if env._states[-1] not in [-2, -3]:
                            env._states.append(-2)
                        raise

                    env._states.append(0)
                    return ret

            return iter_get_prediction_days()

        @staticmethod
        def predict(p):
            env = __class__
            if env._states[-1] == 1:
                raise env.KaggleEnvError('must get next prediction day before submitting a prediction again')
            elif env._states[-1] == -1:
                raise env.KaggleEnvError('must get next prediction day before submitting a prediction')
            elif env._states[-1] in [-2, -3]:
                raise env.KaggleEnvError('cannot submit prediction, every prediction has already been submitted')
            elif env._states[-1] != 0:
                raise env.KaggleEnvError('environment internal error [bad state {}]'.format(env._states[-1]))
            env._submitted.append(p)
            env._states.append(1)

        @staticmethod
        def write_submission_file():
            env = __class__
            if env._states[-1] not in [-2, -3]:
                raise env.KaggleEnvError('must be finished predicting before writing submission file')

            template = pd.concat((day[2] for day in test), axis=0)
            try:
                submission = pd.concat((sub for sub in env._submitted), axis=0)
                assert (template.shape==submission.shape), 'submissions have malformed shape'
                assert (template.index==submission.index).all(), 'submissions have malformed index'
                assert (template.columns==submission.columns).all(), 'submissions have malformed columns'
                assert (template.assetCode==submission.assetCode).all(), 'submissions have malformed assetCode column'
            except Exception as e:
                raise env.KaggleEnvError(e)

            env._submission = submission
            if env._states[-1] != -3:
                env._states.append(-3)
            print('''Your submission file has been saved. Once you `Commit` your Kernel and it finishes running, '''+
                    '''you can submit the file to the competition from the Kernel Viewer `Output` tab.''')
    return __env__

In [7]:
__env__ = _make_env()

# #TODO #KILL everything above this here ^^^^^^^^^^^^^^^^^^^^

# +++++++++++++++++++++ production additions ++++++++++++++++++++++

### imports

In [8]:
import numpy as np
import pandas as pd

import random, math, functools, itertools, os, gc
from collections import Counter, namedtuple
from itertools import product

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
#import altair as alt
#from altair import *
#sns.set()
plt.style.use(['classic', 'seaborn', 'seaborn-colorblind'])

def canvas(width, height):
    dpi = 96
    from pylab import rcParams
    rcParams['figure.figsize'] = width, height
    
import lightgbm as lgb
import numba
from numba import njit

### `utility.py`

In [9]:
from itertools import chain
from importlib import reload
from functools import wraps, reduce, lru_cache
memoized = lru_cache(maxsize=-1, typed=False)

In [10]:
def inside(cls, name=None):
    '''decorator with one argument that sets the decorated as an attribute of the argument.
    also optional argument `name` exists for assigning things that don't automatically get __name__ attribute attached'''
    def put_inside(a):
        nonlocal name
        name = name if name is not None else a.__name__
        setattr(cls, name, a)
        return a
    return put_inside

### `object.py`

In [11]:
# This cell implements Namespace objects. Read the README here https://github.com/yumichael/pytil

import collections

def _mydict_init_items_(*args, **kwargs):
    items = ()
    if len(args) == 3:
        space = args[2]
        for k in list(space.keys()):
            if str.startswith(k, '__') and str.endswith(k, '__'):
                del space[k]
        items = space.items()
    elif args and isinstance(args[0], collections.Mapping):
        items = args[0].items()
    elif args and isinstance(args[0], collections.Iterable):
        items = args[0]
    if kwargs:
        items = chain(items, kwargs.items())
    return items

def _mydict_pretty_factory_(start='{', end='}', relater=': ', delimiter=',',
                            key_action=lambda p, k: p.pretty(k), base=None):
    def _repr_pretty_(obj, p, cycle):
        nonlocal start, end, relater, delimiter, key_action, base
        typ = type(obj)

        beginning = typ.__name__ + '(' + start
        ending = end + ")"

        if typ is not base and typ.__repr__ != base.__repr__:
            # If the subclass provides its own repr, use it instead.
            return p.text(typ.__repr__(obj))

        if cycle:
            return p.text(beginning + '...' + ending)
        p.begin_group(1, beginning)
        keys = typ.keys(obj)
        # if dict isn't large enough to be truncated,
        #   sort keys before displaying
        if not (p.max_seq_length and len(obj) >= p.max_seq_length):
            try:
                keys = sorted(keys)
            except Exception:
                # Sometimes the keys don't sort.
                pass
        for idx, key in p._enumerate(keys):
            if idx:
                p.text(delimiter)
                p.breakable()
            key_action(p, key)
            p.text(relater)
            p.pretty(obj[key])
        p.end_group(1, ending)
    return _repr_pretty_

def add_mydict_pprinter(*args, **kwargs):
    def decorator(cls):
        kwds = dict(kwargs)
        kwds['base'] = cls
        cls._repr_pretty_ = _mydict_pretty_factory_(*args, **kwds)
        return cls
    return decorator


class DefaultSlots(type):
    def __new__(meta, name, bases, attrs):
        if '__slots__' not in attrs:
            attrs['__slots__'] = ()
        return super().__new__(meta, name, bases, attrs)

@add_mydict_pprinter()
class DictObject(dict, metaclass=DefaultSlots):
    def __init__(self, *args, **kwargs):
        data = ((k, v) for k, v in _mydict_init_items_(*args, **kwargs))
        super().__init__(data)
    
    def __repr__(self):
        return "{}({})".format(type(self).__name__, super().__repr__())

    
    def __getattribute__(self, name):
        try:
            return self[name]
        except KeyError:
            errstr = "'{}' object has no attribute '{}'"
            raise AttributeError(errstr.format(type(self).__name__, name))

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        try:
            del self[name]
        except KeyError:
            raise AttributeError(name)

@add_mydict_pprinter('', '', '=', ',', lambda p, k: p.text(k))
class Namespace(DictObject):
    def __init__(self, *args, **kwargs):
        items = _mydict_init_items_(*args, **kwargs)
        data = ((k, v) for k, v in items if isinstance(k, str))
        super(__class__, type(self)).__init__(self, data)

    def __setattr__(self, name, value):
        if not isinstance(name, str):
            errstr = "{} is not a valid attribute name identifier"
            raise ValueError(errstr.format(name))
        self[name] = value

    def __repr__(self):
        attach = __class__.__repr__
        is_orig = False
        if not hasattr(attach, '_seen'):
            is_orig = True
            attach._seen = {}
        elif id(self) in attach._seen:
            return type(self).__name__ + "(...)"
        attach._seen[id(self)] = self
        try:
            it = type(self).items(self)
            body = ', '.join("{}={}".format(a, repr(v)) for a, v in it)
            return "{}({})".format(type(self).__name__, body)
        finally:
            del attach._seen[id(self)]
            if is_orig:
                del attach._seen

    def mycopy(self, copied={}):
        is_orig = not copied
        selfc = copied[id(self)] = type(self)()
        for key, obj in type(self).items(self):
            if isinstance(obj, __class__):
                if id(obj) in copied:
                    selfc[key] = copied[id(obj)]
                else:
                    selfc[key] = type(obj).mycopy(obj, copied=copied)
            else:
                selfc[key] = obj
        if is_orig:
            copied.clear()
        return selfc
    
    py = lambda o: {str(k): v for k, v in dict.items(o)}

def copy(obj):
    assert isinstance(obj, Namespace)
    return type(obj).mycopy(obj)

########################################################################
O = Namespace
Namespace.__name__ = '<>'

class NamedNamespace(Namespace):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if len(args) == 3:
            self.__name__ = args[0]
    def __repr__(self):
        attach = __class__.__repr__
        is_orig = False
        if not hasattr(attach, '_seen'):
            is_orig = True
            attach._seen = {}
        elif id(self) in attach._seen:
            return f"<{self.__name__}>(...)"
        attach._seen[id(self)] = self
        try:
            it = type(self).items(self)
            body = ', '.join("{}={}".format(a, repr(v)) for a, v in it if a != '__name__')
            return f"<{self.__name__}>({body})"
        finally:
            del attach._seen[id(self)]
            if is_orig:
                del attach._seen
NamedO = NamedNamespace
NamedO.__name__ = '<>'

### `combos.py`

In [12]:
# mostly ignore this cell, it's only here because it would be annoying to rework one bit of feature generation code to not
# use the code here

import itertools
import functools
import math
import operator


def binom(n, k):
    k = min(k, n - k)
    return functools.reduce(lambda a, b: a * (n - b) // (b + 1), range(k), 1)


@memoized
def consecutive_1s(n):
    s = n * (n + 1) // 2
    c = np.zeros([s, n], dtype=np.int8)
    t = 0
    for i in range(n):
        for j in range(i + 1, n + 1):
            for k in range(i, j):
                c[t, k] = 1
            t += 1
    return c


@memoized
def one_minus_one(n):
    s = n * (n - 1) // 2
    c = np.zeros([s, n], dtype=np.int8)
    t = 0
    for i in range(n):
        for j in range(i + 1, n):
            c[t, i] = 1
            c[t, j] = -1
            t += 1
    return c


@memoized
def one_minus_one_skip_by_k(n, k):
    m = math.ceil(n / k)
    s = m * (m - 1) // 2
    c = np.zeros([s, n], dtype=np.int8)
    t = 0
    for i in range(0,n,k):
        for j in range(i + k, n, k):
            c[t, i] = 1
            c[t, j] = -1
            t += 1
    return c




@memoized
def int_coefficients_mod_sign(n, m, duplicates=True):
    guys = []
    path = [0]
    while True:
        while True:
            if not path:
                return np.stack(guys, axis=0) if guys else np.ndarray([0, n], dtype=np.int8)
            path[-1] += 1
            if path[-1] == 0:
                path[-1] += 1
            if path[-1] > m:
                path.pop()
                continue
            if len(path) == n:
                break
            path.append(-m - 1)
        if duplicates or functools.reduce(math.gcd, path) == 1:
            guys.append(np.asarray(path, dtype=np.int8))


@memoized
def up_to_m_int_coefficients_mod_sign(n, m, h, duplicates=True):
    guys = []
    ident = np.eye(n, dtype=np.int8)
    for k in range(1, m + 1):
        for x_list in itertools.combinations(ident, k):
            for sc in int_coefficients_mod_sign(k, h, duplicates):
                guy = sum(x * b for b, x in zip(sc, x_list))
                guys.append(guy)
    return np.stack(guys, axis=0).astype(np.int8, copy=False)


@memoized
def consecutive_k_1s(m,k):
    assert m%k==0, 'consecutive_k_1s argument m must be divisible by argument k'
    n = m // k
    s = n * (n + 1) // 2
    c = np.zeros([s, k*n], dtype=np.int8)
    t = 0
    for i in range(0,k*n,k):
        for j in range(i + k, k*n + 1, k):
            for l in range(i, j):
                c[t, l] = 1
            t += 1
    return c


class name(metaclass=O):
    def index(c):
        assert len(c) < 16
        pos, neg = [], []
        for i, a in enumerate(c):
            assert isinstance(a, np.integer) or isinstance(a, int)
            if a != 0:
                arr = pos if a > 0 else neg
                arr += [i] * abs(a)
        strpos = ''.join(hex(i)[2:] for i in pos)
        strneg = ''.join(hex(i)[2:] for i in neg)
        if not neg:
            return strpos
        else:
            return strpos+'/'+strneg
    def consec(c):
        assert c.any() and (c!=np.shift(c,-1))[:-1].sum()<=2 # c switches from 0 to something then to 0, or switch less
        nonzero = c.nonzero()[0]
        i, j = nonzero[0], nonzero[-1]
        return '({}..{})^{}'.format(i,j,c[i])


def union(*cbs):
    assert all(cb.shape[1] == cbs[0].shape[1] for cb in cbs)
    seen, the = set(), []
    for cb in cbs:
        for c in cb:
            if tuple(c) not in seen:
                the.append(c)
                seen.update([tuple(c), tuple(-c)])
    return np.stack(the)
    
    
class combos(metaclass=O):
    name = name
    union = union
    c1co = consecutive_1s #(n)
    ckco = consecutive_k_1s #(n)
    mintco = lambda n, m, h: up_to_m_int_coefficients_mod_sign(n, m, h, False) #(n, m=<max # !=0 coefs>, <max |coef|>)
    omo = one_minus_one
    omok = one_minus_one_skip_by_k

### `helper.ipynb`

In [13]:
def lens(*it):
    return tuple(len(x) for x in it)
def shapes(*arrs):
    return tuple(x.shape for x in arrs)
def sums(*it):
    return tuple(sum(x) for x in it)

In [14]:
def shift(a, shift, axis=None):
    '''Params - a: np.ndarray
                shift: integer, + means entries moved to greater indices, - means entries moved to smaller indices
                axis: one integer
    Equivalent to pd.DataFrame.shift
    '''
    if axis is None:
        assert len(a.shape)==1, "Only an array of single dimension can be shifted without specifying axis"
        axis = 0
    assert isinstance(axis, int) and isinstance(shift, int)
    if not shift:
        return a
    padding = (shift,0) if shift>0 else (0,-shift)
    slicing = slice(None,-shift) if shift>0 else slice(-shift,None)
    n = len(a.shape)
    axis = n+axis if axis<0 else axis
    ret = np.pad(a, ((0,0),)*axis+(padding,)+((0,0),)*(n-axis-1), mode='constant', constant_values=(np.nan,))
    ret = ret[(slice(None),)*axis + (slicing,) + (slice(None),)*(n-axis-1)]
    return ret
np.shift = shift

In [87]:
def print_progress(i, dot=1, numdot=10, print=print):
    '''prints dots (.) on loop index at `dot` intervals and prints the actual index on `dot*numdot` intervals'''
    print('.' if i // dot % numdot else i, end='', flush=True) if i % dot == 0 else None

In [16]:
offdiag = lambda a: a[np.where(~np.eye(a.shape[0],dtype=bool))]
nonnans = lambda a: a[np.isfinite(a)]
plist = lambda a, **kw: print('['+','.join(str(x) for x in a)+']', **kw)

In [17]:
def rolling_window_axis0(a, size, step=1):
    '''backward rolling window only
    windows dimension will be appended to a.shape (available at index -1)'''
    a_ext = np.concatenate(( np.full((step*(size-1),)+a.shape[1:],np.nan), a), axis=0)
    n = a_ext.strides[0]
    strided = np.lib.stride_tricks.as_strided     
    return strided(a_ext, shape=a.shape+(size,), strides=a_ext.strides+(step*n,))

In [18]:
def only_nans(a):
    '''Assume `a` either pd.DataFrame, pd.Series, or np.ndarray
    `a` but with all non-NaN values replaced with 0'''
    if isinstance(a, pd.DataFrame) or isinstance(a, pd.Series):
        return a.isna().replace(True, np.nan)
    else:
        a = a.copy()
        a[~np.isnan(a)] = 0
        return a

In [19]:
def wmean(a, weights, axis=0):
    '''Params - a: either a {observations}x{features} `DataFrame`/`ndarray` or a 1D `ndarray`
                weights: weights of same shape as `a` or else broadcastable to `a`
    Weighted mean, with `a.shape` minus the {observations} dimension.'''
    w = weights + only_nans(a) #broadcast(weights, only_nans(a), '+')
    return np.nansum(a * w, axis=axis) / np.nansum(w, axis=axis)
pd.DataFrame.wmean = wmean
pd.Series.wmean = wmean

In [20]:
class staticclass(type):
    '''The staticclass metatype. Use as a metaclass. Will make your class methods static by default.'''
    class InstantiationError(Exception):
        pass
    def __new__(cls, name, parents, attrs):
        for k, v in list(dict.items(attrs)):
            if not isinstance(v, staticmethod) and callable(v):
                attrs[k] = staticmethod(v)
        return super(__class__, __class__).__new__(cls, name, parents, attrs)
    def __call__(cls, *a, **k):
        raise __class__.InstantiationError('cannot instantiate a staticclass')

In [21]:
def ffill_axis0(arr):
    '''Forward fill your np.ndarray along the index 0 axis'''
    n = len(arr.shape)
    mask = np.isnan(arr)
    idx = np.where(~mask, np.arange(mask.shape[0])[(slice(None),)+(np.newaxis,)*(n-1)], 0)
    np.maximum.accumulate(idx,axis=0, out=idx)
    out = arr[(idx,) + tuple(
        np.arange(idx.shape[i])[(np.newaxis,)*i+(slice(None),)+(np.newaxis,)*(n-1-i)] for i in range(1, n)
    ) ]
    return out

In [22]:
stringify_list = lambda a: '["'+'","'.join(str(x) for x in a)+'"]'

don't forget the others didn't import in `helper.py`

### helper code but only for this project

In [23]:
"""Bisection algorithms. This is literally just copied from Python Library's bisect module, but numba decorated."""
@njit
def bisect_left(a, x, lo=0, hi=None):
    """Return the index where to insert item x in list a, assuming a is sorted.
    The return value i is such that all e in a[:i] have e < x, and all e in
    a[i:] have e >= x.  So if x already appears in the list, a.insert(x) will
    insert just before the leftmost x already there.
    Optional args lo (default 0) and hi (default len(a)) bound the
    slice of a to be searched.
    """

    if lo < 0:
        raise ValueError('lo must be non-negative')
    if hi is None:
        hi = len(a)
    while lo < hi:
        mid = (lo+hi)//2
        if a[mid] < x: lo = mid+1
        else: hi = mid
    return lo

In [24]:
# this cell's code is for scrambling a categorical (int) column
# basically creates "random numbers". just an idea I had where I wanted to get "random" but deterministic assetCodeIds

@njit
def shiftWrap32(num, shift):
    d = np.uint8(32)
    n = np.uint32(num & 0xFFFFFFFF) # coerce num to a int32 value
    i = np.uint8(shift % d) # coerce shift to a int8 value
    return np.uint32(n << i) | np.uint32(n >> (d-i))

@njit
def randomThing(i, j):
    return (
        ((0x80085111 ^ shiftWrap32(i, j**2)) * 
        #(0xFA100000 - ~shiftWrap32(i, (j+1)**2))) * 
        #((0x1337C0DE + shiftWrap32(i, (j+2)**2)) ^ 
        (0xABCDEF77 ^ shiftWrap32(i, (j+3)**2)))
    )

@memoized
def randMap32(ident):
    i = ident
    @njit
    def func(n):
        s = shiftWrap32
        x = n
        x *= randomThing(i, 777) | 0x5558982A
        x += randomThing(i, 263435)
        x *= x
        x = (x >> 8) + (x << 8)
        x ^= randomThing(i, 2000004444)
        x *= x
        return x
    return func

In [25]:
def convert32(F):
    '''just converts all columns in dataframe `F` into 32-bit dtypes'''
    for c in F.columns:
        if np.issubdtype(F[c].dtype, np.float):
            if F[c].dtype != np.float32:
                F[c] = F[c].astype(np.float32)
        elif np.issubdtype(F[c].dtype, np.integer):
            if F[c].dtype != np.int32:
                F[c] = F[c].astype(np.int32)
        else:
            assert False, 'dtype other than float or int found in features'
    gc.collect()

In [26]:
from types import LambdaType

class Featurespace(Namespace):
    '''This cell defines a Namespace subclass that chooses selectively to evaluate and save its attribute assignments or not.
    It is needed because my feature generation code generates A LOT of features, not all of which I want to use.
    So the `allow` function filters out features I don't want to use, so as to save memory and time.
    
    It has its __setitem__ overloaded so that it either takes the actual value, a function -> actual value, or a tuple
    of (function -> actual value, function -> next row to append to actual value)
    
    Also, some features depend on previous features existing, so there is logic to build the dependent features set.
    '''
    allow = lambda name: True
    depend_graph = {}
    record_dependency = False
    _accessed = set()
    
    @classmethod
    def build_dependency(cls):
        stack = list(filter(cls.allow, cls.depend_graph))
        seen = set(stack)
        cls.depends = set(stack)
        while stack:
            name = stack.pop()
            for next in cls.depend_graph.get(name, frozenset()):
                if next not in seen:
                    stack.append(next)
                    seen.add(next)
                    cls.depends.add(next)
    
    def __getitem__(self, name):
        cls = type(self)
        if cls.record_dependency:
            cls._accessed.add(name)
        return super(__class__, type(self)).__getitem__(self, name)
    
    def __getattribute__(self, name):
        cls = type(self)
        if cls.record_dependency:
            cls._accessed.add(name)
        return super(__class__, type(self)).__getattribute__(self, name)
    
    def __setitem__(self, name, value):
        cls = type(self)
        
        if hasattr(cls, 'depends'):
            care = name in cls.depends
        else:
            care = cls.allow(name)
        if not cls.record_dependency and not care:
            return
        
        if isinstance(value, LambdaType):
            ans = super(__class__, type(self)).__setitem__(self, name, value())
        elif isinstance(value,tuple) and len(value)==2 and all(isinstance(value[i],LambdaType) for i in range(2)):
            if name not in self:
                ans = super(__class__, type(self)).__setitem__(self, name, value[0]())
            else:
                changed = np.concatenate([self[name], value[1]()[np.newaxis]], axis=0)
                ans = super(__class__, type(self)).__setitem__(self, name, changed)
        else:
            ans = super(__class__, type(self)).__setitem__(self, name, value)
            
        if cls.record_dependency:
            assert name not in cls.depend_graph, 'each name in Featurespace should only be assigned once'
            cls.depend_graph[name] = frozenset(cls._accessed)
            cls._accessed = set()
            
        return ans
OX = Featurespace

## %%%%%%%%%%% requirements check for main prediction code: %%%%%%%%%%%

In [27]:
__env__
O
gc
canvas
print("ok go")

ok go


# =================== MAIN PREDICTION CODE ====================

In [28]:
import warnings
warnings.filterwarnings('ignore')

BUG = O() # put stuff in this that you want to see for debugging needs

%matplotlib inline
canvas(12,12)

In [29]:
M, N = __env__.get_training_data()
del N; gc.collect()

0

#### bit of a hack of the previouly LB securing random values

In [30]:
#TODO #KILL? this should not be in the production code
# this was an old feature of the scrambled assetCodeIds that I could not regenerate programmatically any more
dfHack2 = pd.read_csv('dfHack2Str.csv', index_col='assetCodeId',
                      dtype={'assetCodeId':np.int,'assetCodeIdWTFFlogabs':np.float32,'assetCodeIdWTFFsin2':np.float32})

### ~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/ Model Definitions ~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/

In [31]:

#!!!!!!!!!!!!!!!!! BEGIN code that you should not read because it just defines the old InitKaggleGo features
fTransNewfault = ["volume","close","open","assetCodeId","assetNameId","cc","oo","acc","aoo","ccTEN","ooTEN","accTEN","aooTEN","dayOfYear","dayOfWeek","oo1Since21","oo1MaxSince21","oo1Drawdown1Since21","oo1Drawdown5Since21","oo1Drawdown10Since21","oo1MaxSince21{10}","oo1MinSince21","oo1MinSince21{10}","oo1Since62","oo1MaxSince62","oo1Drawdown1Since62","oo1Drawdown5Since62","oo1Drawdown10Since62","oo1MaxSince62{10}","oo1MinSince62","oo1MinSince62{10}","oo1Since125","oo1MaxSince125","oo1Drawdown1Since125","oo1Drawdown5Since125","oo1Drawdown10Since125","oo1MaxSince125{10}","oo1MinSince125","oo1MinSince125{10}","oo1Since250","oo1MaxSince250","oo1Drawdown1Since250","oo1Drawdown5Since250","oo1Drawdown10Since250","oo1MaxSince250{10}","oo1MinSince250","oo1MinSince250{10}","aoo1Since21","aoo1MaxSince21","aoo1Drawdown1Since21","aoo1Drawdown5Since21","aoo1Drawdown10Since21","aoo1MaxSince21{10}","aoo1MinSince21","aoo1MinSince21{10}","aoo1Since62","aoo1MaxSince62","aoo1Drawdown1Since62","aoo1Drawdown5Since62","aoo1Drawdown10Since62","aoo1MaxSince62{10}","aoo1MinSince62","aoo1MinSince62{10}","aoo1Since125","aoo1MaxSince125","aoo1Drawdown1Since125","aoo1Drawdown5Since125","aoo1Drawdown10Since125","aoo1MaxSince125{10}","aoo1MinSince125","aoo1MinSince125{10}","aoo1Since250","aoo1MaxSince250","aoo1Drawdown1Since250","aoo1Drawdown5Since250","aoo1Drawdown10Since250","aoo1MaxSince250{10}","aoo1MinSince250","aoo1MinSince250{10}","oo5","oo10","oo15","oo20","oo(10-5)","oo(15-5)","oo(20-5)","oo(15-10)","oo(20-10)","oo(20-15)","aoo5","aoo10","aoo15","aoo20","aoo(10-5)","aoo(15-5)","aoo(20-5)","aoo(15-10)","aoo(20-10)","aoo(20-15)","(it,af,it{1},aoo,acc){3/4}"]
fTransNewnewfault = fTransNewfault[:-1] + ['(aoo-acc)']
fTime = 'dayOfYear dayOfWeek'.split()
fAsset = 'assetCodeId assetNameId'.split()
fPriceAbsolute = 'close open'.split()
def select_relative(f):
    return f not in set(fTime+fAsset+fPriceAbsolute)
fInitKaggleGo = list(filter(select_relative, fTransNewnewfault)) + ['assetCodeIdWTFFlogabs', 'assetCodeIdWTFFsin2']
# _fMust = ['oo','(oo-aoo)','(cc-acc)','(oo-cc)','(aoo-acc)','vp1dd']
# fMustInclude = _fMust+[c+'1' for c in _fMust]+[c+'*' for c in _fMust]
# fMustInclude += 'af it aoo acc'.split()
class ikg(NamedO()):
    features = fInitKaggleGo
    train_on = [2009.,2009.75,2010.75,2011.5,2012.25,2012.5,2012.75,2013.,2013.25,2013.75,2014.,2014.25,2015.]
    weight = 'flat_weight'
    params = dict(early_stopping_round=50,learning_rate=.05,num_leaves=1<<12,max_depth=12,min_data_in_leaf=150,min_sum_hessian_in_leaf=50,lambda_l1=.01,lambda_l2=.01)
_ikg = ikg; del ikg
#!!!!!!!!!!!!!!!!!!!!!! END code that you should not read

fullhouse = [2010.,2010.25,2010.5,2010.75,
             2011.,2011.25,2011.5,2011.75,
             2012.,2012.25,2012.5,2012.75,
             2013.,2013.25,2013.5,2013.75,
             2014.,2014.25,2014.5,2014.75,
             2015.,2015.25,#2015.5,2015.75,
             #2016.,2016.25,2016.5,2016.75,
             ]

class bstdefs(O()):
    '''Each item under here fully defines a model
        features = the list of feature names
        train_on = the quarters that LightGBM should train on
        weight = the name of the weight to use
        params = LGBM params (in addition to default)
    '''
    ikg = _ikg # the InitKaggleGo model
    class IKGCover(NamedO()):
        features = ["assetCodeId","volume","cc","oo","acc","aoo","oo5","oo10","oo15","aoo5","aoo10","aoo15","af","it","rr","oo(10-5)","oo(15-10)","oo(20-15)","oo(15-5)","oo(20-10)","oo(20-5)","aoo(10-5)","aoo(15-10)","aoo(20-15)","aoo(15-5)","aoo(20-10)","aoo(20-5)","af{1}","it{1}","(it,af,it{1},aoo,acc){0/1}","(it{1},af{1},it{2},aoo{1},acc{1}){0/1}","(it,af,it{1},aoo,acc){0/2}","(it{1},af{1},it{2},aoo{1},acc{1}){0/2}","(it,af,it{1},aoo,acc){0/3}","(it{1},af{1},it{2},aoo{1},acc{1}){0/3}","(it,af,it{1},aoo,acc){0/4}","(it{1},af{1},it{2},aoo{1},acc{1}){0/4}","(it,af,it{1},aoo,acc){1/2}","(it{1},af{1},it{2},aoo{1},acc{1}){1/2}","(it,af,it{1},aoo,acc){1/3}","(it{1},af{1},it{2},aoo{1},acc{1}){1/3}","(it,af,it{1},aoo,acc){1/4}","(it{1},af{1},it{2},aoo{1},acc{1}){1/4}","(it,af,it{1},aoo,acc){2/3}","(it{1},af{1},it{2},aoo{1},acc{1}){2/3}","(it,af,it{1},aoo,acc){2/4}","(it{1},af{1},it{2},aoo{1},acc{1}){2/4}","oo1Since21","oo1MaxSince21","oo1Drawdown1Since21","oo1Drawdown5Since21","oo1Drawdown10Since21","oo1MinSince21","oo1MinSince21{10}","aoo1Since21","aoo1MaxSince21","aoo1Drawdown1Since21","aoo1Drawdown5Since21","aoo1Drawdown10Since21","aoo1MinSince21","aoo1MinSince21{10}","oo1Since62","oo1MaxSince62","oo1Drawdown1Since62","oo1Drawdown5Since62","oo1Drawdown10Since62","oo1MinSince62","oo1MinSince62{10}","aoo1Since62","aoo1MaxSince62","aoo1Drawdown1Since62","aoo1Drawdown5Since62","aoo1Drawdown10Since62","aoo1MinSince62","aoo1MinSince62{10}","oo1Since125","oo1MaxSince125","oo1Drawdown1Since125","oo1Drawdown5Since125","oo1Drawdown10Since125","oo1MinSince125","oo1MinSince125{10}","aoo1Since125","aoo1MaxSince125","aoo1Drawdown1Since125","aoo1Drawdown5Since125","aoo1Drawdown10Since125","aoo1MinSince125","aoo1MinSince125{10}","oo1Since250","oo1MaxSince250","oo1Drawdown1Since250","oo1Drawdown5Since250","oo1Drawdown10Since250","oo1MinSince250","oo1MinSince250{10}","aoo1Since250","aoo1MaxSince250","aoo1Drawdown1Since250","aoo1Drawdown5Since250","aoo1Drawdown10Since250","aoo1MinSince250","aoo1MinSince250{10}","assetCode_inUniCount"]
        feats_ctor = '\n    <>~Market & (\n        <>Return{.&pure&~digit&~TEN} & ~Since & Return{.&\n            <>(oo|cc|aoo|acc|daoc)\n            & index[0]\n          }\n        | Return{.&pure&~index} & ~Since & Return{.&\n            <>(oo|aoo)\n            & [5:,10:,15:,10:5,15:10,20:15,15:5,20:10,20:5]\n          }\n        | (\n            <>Return{(oo{.&[1:]}|aoo[1:])}\n            & (\n                <> ~Drawdown&~Drawup&~Since{Min}&~Since{Max}\n                | Drawdown[1:, 5:,10:]\n                | Since{Max&index[0]}\n                | Since{Min&index[0,10]}\n              )\n            & Since[21:,62:,125:,250:]\n          )\n      )\n    |\n        <>AssetEnc{InUni}\n        | AssetEnc{Code}\n    |\n        <>FaceValue{Volume}\n        | Return{mix[1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19]}\n        | Return{rr|af|it}\n    '
        train_on = fullhouse
        weight = 'flat_weight'
        params = dict(num_iterations=450,learning_rate=0.05,max_depth=-1,num_leaves=4096,min_data_in_leaf=120,min_sum_hessian_in_leaf=100,lambda_l1=0,lambda_l2=0)
    class IKGCoverHomo(NamedO()):
        features = ["volume","cc","oo","acc","aoo","oo5","oo10","oo15","aoo5","aoo10","aoo15","af","it","rr","oo(10-5)","oo(15-10)","oo(20-15)","oo(15-5)","oo(20-10)","oo(20-5)","aoo(10-5)","aoo(15-10)","aoo(20-15)","aoo(15-5)","aoo(20-10)","aoo(20-5)","af{1}","it{1}","(it,af,it{1},aoo,acc){0/1}","(it{1},af{1},it{2},aoo{1},acc{1}){0/1}","(it,af,it{1},aoo,acc){0/2}","(it{1},af{1},it{2},aoo{1},acc{1}){0/2}","(it,af,it{1},aoo,acc){0/3}","(it{1},af{1},it{2},aoo{1},acc{1}){0/3}","(it,af,it{1},aoo,acc){0/4}","(it{1},af{1},it{2},aoo{1},acc{1}){0/4}","(it,af,it{1},aoo,acc){1/2}","(it{1},af{1},it{2},aoo{1},acc{1}){1/2}","(it,af,it{1},aoo,acc){1/3}","(it{1},af{1},it{2},aoo{1},acc{1}){1/3}","(it,af,it{1},aoo,acc){1/4}","(it{1},af{1},it{2},aoo{1},acc{1}){1/4}","(it,af,it{1},aoo,acc){2/3}","(it{1},af{1},it{2},aoo{1},acc{1}){2/3}","(it,af,it{1},aoo,acc){2/4}","(it{1},af{1},it{2},aoo{1},acc{1}){2/4}","oo1Since21","oo1MaxSince21","oo1Drawdown1Since21","oo1Drawdown5Since21","oo1Drawdown10Since21","oo1MinSince21","oo1MinSince21{10}","aoo1Since21","aoo1MaxSince21","aoo1Drawdown1Since21","aoo1Drawdown5Since21","aoo1Drawdown10Since21","aoo1MinSince21","aoo1MinSince21{10}","oo1Since62","oo1MaxSince62","oo1Drawdown1Since62","oo1Drawdown5Since62","oo1Drawdown10Since62","oo1MinSince62","oo1MinSince62{10}","aoo1Since62","aoo1MaxSince62","aoo1Drawdown1Since62","aoo1Drawdown5Since62","aoo1Drawdown10Since62","aoo1MinSince62","aoo1MinSince62{10}","oo1Since125","oo1MaxSince125","oo1Drawdown1Since125","oo1Drawdown5Since125","oo1Drawdown10Since125","oo1MinSince125","oo1MinSince125{10}","aoo1Since125","aoo1MaxSince125","aoo1Drawdown1Since125","aoo1Drawdown5Since125","aoo1Drawdown10Since125","aoo1MinSince125","aoo1MinSince125{10}","oo1Since250","oo1MaxSince250","oo1Drawdown1Since250","oo1Drawdown5Since250","oo1Drawdown10Since250","oo1MinSince250","oo1MinSince250{10}","aoo1Since250","aoo1MaxSince250","aoo1Drawdown1Since250","aoo1Drawdown5Since250","aoo1Drawdown10Since250","aoo1MinSince250","aoo1MinSince250{10}"]
        feats_ctor = '\n    <>~Market & (\n        <>Return{.&pure&~digit&~TEN} & ~Since & Return{.&\n            <>(oo|cc|aoo|acc|daoc)\n            & index[0]\n          }\n        | Return{.&pure&~index} & ~Since & Return{.&\n            <>(oo|aoo)\n            & [5:,10:,15:,10:5,15:10,20:15,15:5,20:10,20:5]\n          }\n        | (\n            <>Return{(oo{.&[1:]}|aoo[1:])}\n            & (\n                <> ~Drawdown&~Drawup&~Since{Min}&~Since{Max}\n                | Drawdown[1:, 5:,10:]\n                | Since{Max&index[0]}\n                | Since{Min&index[0,10]}\n              )\n            & Since[21:,62:,125:,250:]\n          )\n      )\n    |\n        <>FaceValue{Volume}\n        | Return{mix[1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19]}\n        | Return{rr|af|it}\n    '
        train_on = fullhouse
        weight = 'flat_weight'
        params = dict(num_iterations=450,learning_rate=0.05,max_depth=-1,num_leaves=4096,min_data_in_leaf=120,min_sum_hessian_in_leaf=100,lambda_l1=0,lambda_l2=0)
    class pIKGCoverHomo(NamedO()):
        Pair = O(hi=None, lo=(.6,500), repeat=3)
        copy_features = ["volume","cc","oo","acc","aoo","oo5","oo10","oo15","aoo5","aoo10","aoo15","af","it","rr","oo(10-5)","oo(15-10)","oo(20-15)","oo(15-5)","oo(20-10)","oo(20-5)","aoo(10-5)","aoo(15-10)","aoo(20-15)","aoo(15-5)","aoo(20-10)","aoo(20-5)","af{1}","it{1}","(it,af,it{1},aoo,acc){0/1}","(it{1},af{1},it{2},aoo{1},acc{1}){0/1}","(it,af,it{1},aoo,acc){0/2}","(it{1},af{1},it{2},aoo{1},acc{1}){0/2}","(it,af,it{1},aoo,acc){0/3}","(it{1},af{1},it{2},aoo{1},acc{1}){0/3}","(it,af,it{1},aoo,acc){0/4}","(it{1},af{1},it{2},aoo{1},acc{1}){0/4}","(it,af,it{1},aoo,acc){1/2}","(it{1},af{1},it{2},aoo{1},acc{1}){1/2}","(it,af,it{1},aoo,acc){1/3}","(it{1},af{1},it{2},aoo{1},acc{1}){1/3}","(it,af,it{1},aoo,acc){1/4}","(it{1},af{1},it{2},aoo{1},acc{1}){1/4}","(it,af,it{1},aoo,acc){2/3}","(it{1},af{1},it{2},aoo{1},acc{1}){2/3}","(it,af,it{1},aoo,acc){2/4}","(it{1},af{1},it{2},aoo{1},acc{1}){2/4}","oo1Since21","oo1MaxSince21","oo1Drawdown1Since21","oo1Drawdown5Since21","oo1Drawdown10Since21","oo1MinSince21","oo1MinSince21{10}","aoo1Since21","aoo1MaxSince21","aoo1Drawdown1Since21","aoo1Drawdown5Since21","aoo1Drawdown10Since21","aoo1MinSince21","aoo1MinSince21{10}","oo1Since62","oo1MaxSince62","oo1Drawdown1Since62","oo1Drawdown5Since62","oo1Drawdown10Since62","oo1MinSince62","oo1MinSince62{10}","aoo1Since62","aoo1MaxSince62","aoo1Drawdown1Since62","aoo1Drawdown5Since62","aoo1Drawdown10Since62","aoo1MinSince62","aoo1MinSince62{10}","oo1Since125","oo1MaxSince125","oo1Drawdown1Since125","oo1Drawdown5Since125","oo1Drawdown10Since125","oo1MinSince125","oo1MinSince125{10}","aoo1Since125","aoo1MaxSince125","aoo1Drawdown1Since125","aoo1Drawdown5Since125","aoo1Drawdown10Since125","aoo1MinSince125","aoo1MinSince125{10}","oo1Since250","oo1MaxSince250","oo1Drawdown1Since250","oo1Drawdown5Since250","oo1Drawdown10Since250","oo1MinSince250","oo1MinSince250{10}","aoo1Since250","aoo1MaxSince250","aoo1Drawdown1Since250","aoo1Drawdown5Since250","aoo1Drawdown10Since250","aoo1MinSince250","aoo1MinSince250{10}"]
        diff_features = ["volume","cc","oo","acc","aoo","oo5","oo10","oo15","aoo5","aoo10","aoo15","af","it","rr","oo(10-5)","oo(15-10)","oo(20-15)","oo(15-5)","oo(20-10)","oo(20-5)","aoo(10-5)","aoo(15-10)","aoo(20-15)","aoo(15-5)","aoo(20-10)","aoo(20-5)","af{1}","it{1}","(it,af,it{1},aoo,acc){0/1}","(it{1},af{1},it{2},aoo{1},acc{1}){0/1}","(it,af,it{1},aoo,acc){0/2}","(it{1},af{1},it{2},aoo{1},acc{1}){0/2}","(it,af,it{1},aoo,acc){0/3}","(it{1},af{1},it{2},aoo{1},acc{1}){0/3}","(it,af,it{1},aoo,acc){0/4}","(it{1},af{1},it{2},aoo{1},acc{1}){0/4}","(it,af,it{1},aoo,acc){1/2}","(it{1},af{1},it{2},aoo{1},acc{1}){1/2}","(it,af,it{1},aoo,acc){1/3}","(it{1},af{1},it{2},aoo{1},acc{1}){1/3}","(it,af,it{1},aoo,acc){1/4}","(it{1},af{1},it{2},aoo{1},acc{1}){1/4}","(it,af,it{1},aoo,acc){2/3}","(it{1},af{1},it{2},aoo{1},acc{1}){2/3}","(it,af,it{1},aoo,acc){2/4}","(it{1},af{1},it{2},aoo{1},acc{1}){2/4}","oo1Since21","oo1MaxSince21","oo1Drawdown1Since21","oo1Drawdown5Since21","oo1Drawdown10Since21","oo1MinSince21","oo1MinSince21{10}","aoo1Since21","aoo1MaxSince21","aoo1Drawdown1Since21","aoo1Drawdown5Since21","aoo1Drawdown10Since21","aoo1MinSince21","aoo1MinSince21{10}","oo1Since62","oo1MaxSince62","oo1Drawdown1Since62","oo1Drawdown5Since62","oo1Drawdown10Since62","oo1MinSince62","oo1MinSince62{10}","aoo1Since62","aoo1MaxSince62","aoo1Drawdown1Since62","aoo1Drawdown5Since62","aoo1Drawdown10Since62","aoo1MinSince62","aoo1MinSince62{10}","oo1Since125","oo1MaxSince125","oo1Drawdown1Since125","oo1Drawdown5Since125","oo1Drawdown10Since125","oo1MinSince125","oo1MinSince125{10}","aoo1Since125","aoo1MaxSince125","aoo1Drawdown1Since125","aoo1Drawdown5Since125","aoo1Drawdown10Since125","aoo1MinSince125","aoo1MinSince125{10}","oo1Since250","oo1MaxSince250","oo1Drawdown1Since250","oo1Drawdown5Since250","oo1Drawdown10Since250","oo1MinSince250","oo1MinSince250{10}","aoo1Since250","aoo1MaxSince250","aoo1Drawdown1Since250","aoo1Drawdown5Since250","aoo1Drawdown10Since250","aoo1MinSince250","aoo1MinSince250{10}"]
        sign_features = []
        corr_features = []
        feats_ctor = '\n    <>~Market & (\n        <>Return{.&pure&~digit&~TEN} & ~Since & Return{.&\n            <>(oo|cc|aoo|acc|daoc)\n            & index[0]\n          }\n        | Return{.&pure&~index} & ~Since & Return{.&\n            <>(oo|aoo)\n            & [5:,10:,15:,10:5,15:10,20:15,15:5,20:10,20:5]\n          }\n        | (\n            <>Return{(oo{.&[1:]}|aoo[1:])}\n            & (\n                <> ~Drawdown&~Drawup&~Since{Min}&~Since{Max}\n                | Drawdown[1:, 5:,10:]\n                | Since{Max&index[0]}\n                | Since{Min&index[0,10]}\n              )\n            & Since[21:,62:,125:,250:]\n          )\n      )\n    |\n        <>FaceValue{Volume}\n        | Return{mix[1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19]}\n        | Return{rr|af|it}\n    '
        train_on = fullhouse
        weight = 'pair_weight'
        params = dict(num_iterations=250,learning_rate=0.05,max_depth=-1,num_leaves=4096,min_data_in_leaf=120,min_sum_hessian_in_leaf=20,lambda_l1=0,lambda_l2=0)
_bd = bstdefs
for name in _bd:
    if 'feats_ctor' in _bd[name]:
        _bd[name].feats_ctor = f"<{name}>"+_bd[name].feats_ctor

def _get_allowed(model):
    if hasattr(model, 'features'):
        return model.features
    else:
        return model.copy_features+model.diff_features+model.sign_features
        
class themodel(O()):
    '''this defines the model blending'''
    solo = O(specs=[_bd.IKGCover], coefs=[1])
    pair = O(specs=[_bd.pIKGCoverHomo], coefs=[.33])
    xAllowed = reduce(operator.__or__,
        [frozenset(_get_allowed(s)) for s,c in chain(zip(solo.specs,solo.coefs), zip(pair.specs,pair.coefs)) if c!=0])

as a note, I sometimes begin a variable name with x, like 'xAllowed', and this just means it's a set (speeds up lookup)

### ~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/~/ END Model Definitions

#### very important ID assignment code

In [33]:
class IdAssign:
    '''This class assigns an integer (>=0) ID to things.
    It automatically updates the count when it sees a new thing without an ID before.
    
    `self.series` gives you a pd.Series where index=ID and the column values give you the original value
    Using `self` as a function maps values to their IDs
    '''
    class NO_MISSING(metaclass=staticclass): # special filler value because `None` might actually be meaningfully used
        pass
    def __init__(self, init, missing, name):
        self.name = name
        self.map = {}
        if missing is not __class__.NO_MISSING:
            self.map[missing] = -1
        i = 0
        for x in init:
            if x not in self.map and x != missing:
                self.map[x] = i
                i += 1
        self.cache = None
    def __call__(self, key):
        if key not in self.map:
            self.map[key] = len(self.map)
        return self.map[key]
    def __len__(self):
        return self.map.__len__()
    @property
    def series(self):
        if self.cache==len(self):
            return self._series
        # Python 3.?+ guarantees that dict keys and values are itered in same order, and that order is insertion order
        self._series = pd.Series(list(self.map.keys()), index=list(self.map.values()), name=self.name)
        self._series.index.name = self.name+'Id'
        self.cache = len(self)
        return self._series

assetCodeSeries = pd.Series(M.assetCode.unique())
assetCodeIdAssign = IdAssign(assetCodeSeries, missing='', name='assetCode')
del assetCodeSeries
assetNameSeries = pd.Series(M.assetName.unique())
assetNameIdAssign = IdAssign(assetNameSeries, missing='Unknown', name='assetName')
del assetNameSeries

#### some data specific helper func/structs

In [34]:
stringify_columns = stringify_list

returns_columns = { # I map these guys from linear space to log space, and make them into shorter names
    'returnsClosePrevRaw1':'cc', 'returnsOpenPrevRaw1':'oo',
    'returnsClosePrevMktres1':'acc', 'returnsOpenPrevMktres1':'aoo',
    'returnsClosePrevRaw10':'ccTEN','returnsOpenPrevRaw10':'ooTEN',
    'returnsClosePrevMktres10':'accTEN','returnsOpenPrevMktres10':'aooTEN'
}
columns_for_U = set(returns_columns.values()) | set(['open', 'close', 'volume'])
excluded_columns = [
    'time','assetCode','assetName','universe','returnsOpenNextMktres10','quarter','y'
]
exclusion_filter = lambda c: c not in excluded_columns
object_columns = ['assetCode', 'assetName']
enumeration_columns = ['assetCodeId', 'assetNameId']

#### hard coded constants

In [35]:
train_start_time = pd.Timestamp('2009-01-01',tz='UTC')
lookback = 60
shortterm = 21
longterm = 250 + lookback*2 #idk it's hard to reason about how much I _really_ need, so this should be pretty safe

#### setting up only the stocks that we have seen in-universe

In [36]:
M['assetCodeId'] = M.assetCode.map(assetCodeIdAssign).astype(int)
inUniCount = M[M.time>=train_start_time].groupby('assetCodeId').universe.sum()
stocksInUni = (inUniCount != 0).pipe(lambda x: x.index[x])
assert stocksInUni.is_monotonic
xStocksInUni = set(stocksInUni) #EDITCELL
inUniCountSeries = inUniCount.astype(float) #EDITCELL
stocksAlways = (inUniCount >= 1981).pipe(lambda x: x.index[x])
xStocksAlways = set(stocksAlways)

### Basic features

In [37]:
def set_basic_features(*,M):
    M['assetCodeId'] = M.assetCode.map(assetCodeIdAssign)#.astype(int) # much more efficient to process everything as float
    del M['assetCode']
    M['assetNameId'] = M.assetName.map(assetNameIdAssign)#.astype(int)
    del M['assetName']
    for orig_col, new_col in returns_columns.items():
        M[new_col] = np.log1p(M[orig_col])
        del M[orig_col]
    # time features
    M['dayOfYear'] = M.time.dt.dayofyear.astype(float)
    M['dayOfWeek'] = M.time.dt.dayofweek.astype(float)

In [91]:
def setup_P(*, F, P, vp=True, always=True):
    P['quarter'] = P.time.dt.year+(P.time.dt.quarter-1)/4
    P['target'] = P.y>0
    P['upDown'] = (P.target*2-1)
    P['absVal'] = np.abs(P.y)
    P['flat_weight'] = P.absVal
    if vp:
        P['vp1_weight'] = P.absVal*F.vp1/1e9
        P['vp5_weight'] = P.absVal*F.vp5/1e9
        P['vp10_weight'] = P.absVal*F.vp10/1e9
        P['vp20_weight'] = P.absVal*F.vp20/1e9
    if always:
        P['always_weight'] = P.absVal*F.assetCodeId.isin(xStocksAlways)

### Making correlations data for pair trading !!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [46]:
@njit
def offdiag_items(ix, x, y):
    m, n = x.shape
    assert (m, n) == y.shape
    assert m == n
    assert (m,) == ix.shape
    t = (n**2-n) // 2
    I = np.full(t, -1, dtype=np.int64)
    J = np.full(t, -1, dtype=np.int64)
    X = np.full(t, 0, dtype=x.dtype)
    Y = np.full(t, 0, dtype=y.dtype)
    k = -1
    for i in range(m):
        for j in range(i+1, n):
            k += 1
            I[k] = ix[i]
            J[k] = ix[j]
            X[k] = x[i,j]
            Y[k] = y[i,j]
    return I, J, X, Y

In [47]:
def make_W_D():
    Mt = M[M.time>=train_start_time]
    set_basic_features(M=Mt)
    ##########################
    W = O()
    W.y1 = Mt[['time','assetCodeId','oo']].pivot('time','assetCodeId').oo
    W.ay1 = Mt[['time','assetCodeId','aoo']].pivot('time','assetCodeId').aoo
    W.y10 = Mt[['time','assetCodeId','ooTEN']].pivot('time','assetCodeId').ooTEN
    W.ay10 = Mt[['time','assetCodeId','aooTEN']].pivot('time','assetCodeId').aooTEN
    W.y20 = W.y10 + W.y10.shift(10)
    W.ay20 = W.ay10 + W.ay10.shift(10)
    W.u = Mt[['time','assetCodeId','universe']].pivot('time','assetCodeId').universe.fillna(0).astype(int)
    # for c in W:
    #     W[c] = W[c][W[c].index>=train_start_time]
    del Mt; gc.collect()
    ##################################
    C = O()
    #C.y1 = W.y1.corr()
    #C.ay1 = W.ay1.corr()
    #C.y10 = W.y10.corr()
    C.ay10 = W.ay10.corr()
    C.u = W.u.T @ W.u
    #############################
    D = O()
    #D.v1 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.y1.index.values, C.y1.values, C.u.values))))
    #D.av1 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.ay1.index.values, C.ay1.values, C.u.values))))
    #D.v10 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.y10.index.values, C.y10.values, C.u.values))))
    D.av10 = pd.DataFrame(dict(zip([0, 1, 'Corr', 'Unic'], offdiag_items(C.ay10.index.values, C.ay10.values, C.u.values))))
    for c in ['av10']:#['v1','av1','v10','av10']:
        D[c] = D[c][D[c].Unic!=0]
        D[c].sort_values('Corr', ascending=False, inplace=True)
    del C; gc.collect()
    return W, D

In [48]:
binary_search = bisect_left

@njit
def index_unique_pairs(A, B, values, repeat):
    assert len(A)==len(B)
    seen = [0] * len(values)
    ans = []
    for i, (a, b) in enumerate(zip(A, B)):
        ia = binary_search(values, a); ib = binary_search(values, b);
        if repeat==-1 or (seen[ia]<repeat and seen[ib]<repeat):
            seen[ia] += 1; seen[ib] += 1;
            ans.append(i)
    return ans

def make_unique_pairs(*, Dl, repeat=-1):
    if repeat==-1:
        return Dl
    ii = index_unique_pairs(Dl[0].values, Dl[1].values, sorted(set(Dl[0].values)|set(Dl[1].values)), repeat=repeat)
    return Dl.iloc[ii]

In [154]:
def make_CC_G_Q_from_Dl_F_P_W(*, Dl, F, P, W, lo, hi=None, repeat=-1,
                              roll_corr=list(product(['ay1','ay10','ay20'],[10,21,62,250]))):
    global TRAIN
    SEE(f'(pair) len(F.columns) = {len(F.columns)}')
    
    D = Dl
    allTime = pd.Series(P.time.unique())
    allTime.index = allTime
    
    # |begin| subset the pairs list
    if hi is None:
        hi = (1., 9999)
    D = D[(lo[0]<=D.Corr)&(D.Corr<=hi[0])&(lo[1]<=D.Unic)&(D.Unic<=hi[1])]
    if isinstance(repeat, int):
        D = D.pipe(lambda x: make_unique_pairs(Dl=x, repeat=repeat))
        w = (D.Corr.fillna(0)*0 + 1.).values
    elif repeat=='prop':
        D01 = D
        D10 = D[[1,0,'Corr','Unic']]
        D2 = pd.concat([D01,D10], axis=0)
        aw = D2.groupby(0).Corr.sum()
        w = (D.Corr / np.maximum(D[0].map(aw), D[1].map(aw))).values
        w /= w.mean()
    else:
        assert False
    if TRAIN:
        hgt = sum(p[-1] for p in D.itertuples())
        CC = np.zeros((hgt,2+len(roll_corr)))
        G0, G1 = np.zeros((hgt,len(F.columns))), np.zeros((hgt,len(F.columns)))
    else:
        CC = []
        G0, G1 = [], []
    Q0, Q1 = [], []
    # |end| subset the pairs list
    
    i = 0
    SEE(f'(pair) generating [{len(D)}]', end='')
    for ii, ((_,a0,a1,corr,unic), weight) in enumerate(zip(D.itertuples(),w)):
        print_progress(ii, dot=10, print=SEE)
        assert len(weight.shape)==0
        BUG.F, BUG.P, BUG.a0, BUG.a1 = F, P, a0, a1
        in0, in1 = (F.assetCodeId==a0)&(P.universe!=0), (F.assetCodeId==a1)&(P.universe!=0)
        A0, A1 = F[in0], F[in1]
        B0, B1 = P[in0], P[in1]
        inTime = set(B0.time)&set(B1.time)
        tm0, tm1 = B0.time.isin(inTime), B1.time.isin(inTime)
        A0, A1 = A0[tm0], A1[tm1]
        B0, B1 = B0[tm0], B1[tm1]
        
        tm = allTime.isin(inTime).values
        def iter_roll_corr():
            for y, r in roll_corr:
                yield W[y][a0].rolling(window=r, min_periods=5).corr(W[y][a1])[tm] #TODO hard coded min_periods
        unic_ = [np.ones_like(W[next(iter(W))].iloc[:,0][tm])*unic]
        weight_ = [np.ones_like(W[next(iter(W))].iloc[:,0][tm])*weight]
        C = np.stack(chain(iter_roll_corr(),unic_,weight_), axis=1)
        
        if TRAIN:
            G0[i:i+unic], G1[i:i+unic] = A0.values, A1.values
            CC[i:i+unic] = C
        else:
            G0.append(A0); G1.append(A1);
            CC.append(C)
        Q0.append(B0); Q1.append(B1);
        
        i += unic
    SEE()
    
    Fcols = F.columns
    del Dl, F, P, W; gc.collect()
        
    Q0, Q1 = pd.concat(Q0, axis=0), pd.concat(Q1, axis=0)
    if TRAIN:
        G0, G1 = pd.DataFrame(G0), pd.DataFrame(G1)
    else:
        G0, G1 = pd.concat(G0, axis=0), pd.concat(G1, axis=0)
        CC = np.concatenate(CC, axis=0)
    G0.index = Q0.index; G1.index = Q1.index;
    G0.columns = Fcols; G1.columns = Fcols;
    CC = pd.DataFrame(CC, columns=[f'__corr__{y}_{r}' for y,r in roll_corr]+['bothInUniCount','weight'])
    return CC, (G0,G1), (Q0,Q1)

In [50]:
def make_FG_PQ_from_CC_G_Q(copy_filter, diff_filter, sign_filter, corr_filter, *, CC, G, Q):
    SEE('(pair) putting together...', end=' ')
    
    weight = CC['weight'].values
    
    Gcols, Qcols = G[0].columns, Q[0].columns
    assert (Gcols==G[1].columns).all() and (Qcols==Q[1].columns).all()
    
    
    cop, dif, sgn = list(filter(copy_filter, Gcols)), list(filter(diff_filter, Gcols)), list(filter(sign_filter, Gcols))
    G0, G1 = G[0][cop].values, G[1][cop].values
    H01 = G[0][dif].values - G[1][dif].values
    I01 = G[0][sgn].values > G[1][sgn].values; I10 = G[1][sgn].values > G[0][sgn].values;
    CC = CC[list(filter(corr_filter, CC.columns))]
    
    F01, F10 = np.concatenate([G0,G1,H01,I01,CC.values], axis=1), np.concatenate([G1,G0,-H01,I10,CC.values], axis=1)
    FG = np.concatenate([F01, F10], axis=0)
    if not np.issubdtype(FG.dtype, np.float):
        FG = FG.astype(np.float32)
    
    FGcols = (['__0__'+c for c in cop] + ['__1__'+c for c in cop] + ['__0-1__'+c for c in dif]
              + ['__0>1__'+c for c in sgn] + list(CC.columns))
    FG = pd.DataFrame(FG, columns=FGcols, copy=False)
    
    
    Q = Q[0].reset_index(drop=True), Q[1].reset_index(drop=True)
    assert(Q[0].time==Q[1].time).all()
    
    Q01 = pd.DataFrame(dict(time=Q[0].time, y=Q[0].y-Q[1].y,
                            **{f'__0__{c}': Q[0][c] for c in ['assetCode','assetName','universe']},
                            **{f'__1__{c}': Q[1][c] for c in ['assetCode','assetName','universe']}))
    Q10 = pd.DataFrame(dict(time=Q[1].time, y=Q[1].y-Q[0].y,
                            **{f'__0__{c}': Q[1][c] for c in ['assetCode','assetName','universe']},
                            **{f'__1__{c}': Q[0][c] for c in ['assetCode','assetName','universe']}))
    
    PQ = pd.concat([Q01, Q10], axis=0, ignore_index=True)
    PQ['pair_weight'] = np.concatenate([weight, weight], axis=0)
    
    PQ.sort_values('time', inplace=True)
    FG = FG.reindex(index=PQ.index, copy=False)
    
    FG.reset_index(drop=True, inplace=True)
    PQ.reset_index(drop=True, inplace=True)
    setup_P(F=None, P=PQ, vp=False, always=False)
    
    SEE('(pair) done')
    return FG, PQ

### THE definitive features generators!

In [51]:
_rollwind = rolling_window_axis0

def noinf_log(a):
    '''np.log, but makes maps 0 => np.nan instead of -np.inf'''
    ret = np.log(a)
    if len(ret.shape)==0:
        return ret if not np.isneginf(ret) else np.nan
    ret[np.isneginf(ret)] = np.nan
    return ret

def _vol(a, axis, fill):
    '''helper that calculates the volatility'''
    var = np.nanvar(a, axis=axis, ddof=1) # ddof=1 for better estimate
    nsamps = np.maximum(np.count_nonzero(~np.isnan(a), axis=axis) - 1, 0)
    # weight by global average to help curb extreme values when cumulating in features constructed from this
    return np.sqrt((var * np.expm1(nsamps) + fill**2) / (np.expm1(nsamps) + 1))
_vol.fill = lambda r: np.sqrt(r) / 22 # these approximate the global average std

Below until the next header are the main feature generation functions.

To save myself time I use single capital letter variables to denote very specific entities

- M = a DataFrame like the original given market obs
- U = an O that maps feature name => np.ndarray of shape ({days up to current day}, {all the stocks, in assetCodeId order})
  - The features are stored in this format to be able to generate lag and drawdown features
- u = an O that maps feature name => np.ndarray of shape ({all the stocks, in assetCodeId order},)
  - These features we don't care about past entries, so only the present day value is included acorss the stocks
- V = an O that maps feature name => np.ndarray of shape ({days up to current day},)
  - These are global (across stocks) features that we need to look back in time for
- v = an O that maps feature name => numbers
  - It's just one number which is the current value of this global feature on the current day

In [52]:
def add_shortterm_price_features(*,U,u):
    # volume*price ############################################################################
    # just the volume*price. it is maximally NaN
    U['vp'] = lambda: U.volume * U.open, lambda: U.volume[-1] * U.open[-1]
    
    # the 1 suffix means there is a best attempt at cleaning up the NaNs and is start of a series of things spanning n days
    U['vp1'] = lambda: ffill_axis0(U['vp']), lambda: ffill_axis0(U['vp'][-2:])[-1]
    # volume*price meaned over h days
    for h in [5,10]:
        U[f'vp{h}'] = lambda: np.nanmean(_rollwind(U.vp1, h), axis=-1), lambda: np.nanmean(U.vp1[-h:], axis=0)
    for h in [20,60]:
        u[f'vp{h}'] = lambda: np.nanmean(U.vp1[-h:], axis=0)
        
    # volume*price day-to-day over h days # 'dd' means "day-to-day". it is the "return" equivalent for volume*price
    for h in [1,5,10]:
        U[f'vp{h}dd1'] = (lambda: noinf_log(U[f'vp{h}']) - noinf_log(np.shift(U[f'vp{h}'],1,axis=0)),
                          lambda: noinf_log(U[f'vp{h}'][-1]) - noinf_log(U[f'vp{h}'][-2]))
    
    #######################
    # just because this wasn't defined for us in `make_single_A_...`
    U['(oo-aoo)1'] = lambda: U.oo1 - U.aoo1
    
    # SHORT TERM COMBOS ######################################################################################
    for ww in ['oo','aoo','(oo-aoo)','vp1dd','vp5dd','volatility']:
        ww1 = ww+'1'
        #! size 5 combos in this section
        if 'olatility' not in ww:
            for i in [5,10,15, 3,6,9,12,18]: # the (i-0) time slices needed by other calcs
                U[ww+str(i)] = lambda: np.nansum(_rollwind(U[ww1],i), axis=-1), lambda: np.nansum(U[ww1][-i:], axis=0)
            for i in [20,60]: # the (i-0) time slices not needed by others
                u[ww+str(i)] = lambda: np.nansum(U[ww1][-i:], axis=0)
        else:
            for r in [5,10,15,20,60, 3,6,9,12,18]:
                U['volatility'+str(r)] = (lambda: _vol(_rollwind(U.oo,r), axis=-1, fill=_vol.fill(r)),
                                          lambda: _vol(U.oo[-r:], axis=0, fill=_vol.fill(r)))
        for r,d in zip([5,10,15, 3,6,9,12,18], [5]*3+[3]*5): # lagged guys
            i = 1
            while r+d*i <= 21:
                u[f'{ww}({r+d*i}-{d*i})'] = lambda: U[f'{ww}{r}'][-1-i*d]
                i += 1
        
    #================================================== LEGACY ============================================================
    T = OX()
    ################################################################# can be NaNs:
    U['(oo-aoo)'] = lambda: U.oo - U.aoo
    U['(cc-acc)'] = lambda: U.cc - U.acc
    U['(oo-cc)'] = lambda: U.oo - U.cc
    U['(aoo-acc)'] = lambda: U.aoo - U.acc
    for ww in ['oo','(oo-aoo)','(cc-acc)','(oo-cc)','(aoo-acc)','vp1dd']:
        T[ww+'Window*'] = lambda: np.flip( (U[ww] if ww in U else U[ww+'*'] if ww+'*' in U else U[ww+'1']) ,axis=0)[:3]
        window = lambda: T[ww+'Window*']
        for c in combos.union(combos.omo(3),combos.mintco(3,1,1)):
            name = '{}{{{}}}'.format(ww, combos.name.index(c))
            if name[-3:]=='{0}':
                name = name[:-3]
            if name not in U and name not in u:
                u[name] = lambda: c@window()
    ################################################################# can be NaNs: # mostly still here for legacy reasons
    log_open, log_close = np.log(U.open), np.log(U.close) # can be NaNs here
    U['af'] = lambda: log_open - np.shift(log_close,1,axis=0)
    U['it'] = lambda: log_close - log_open
    U['rr'] = lambda: log_open - np.shift(log_open,1,axis=0)
    u['af{1}'] = lambda: U.af[-2]
    u['it{1}'] = lambda: U.it[-2]
    
    T['near0*'] = lambda: np.stack([U.it[-1],U.af[-1],U.it[-2],U.aoo[-1],U.acc[-1]], axis=0); near0 = lambda: T['near0*']
    T['near1*'] = lambda: np.stack([U.it[-2],U.af[-2],U.it[-3],U.aoo[-2],U.acc[-2]], axis=0); near1 = lambda: T['near1*']
    for c in combos.omo(5):
        name = '(it,af,it{{1}},aoo,acc){{{}}}'.format(combos.name.index(c))
        u[name] = lambda: c@near0()
        name = '(it{{1}},af{{1}},it{{2}},aoo{{1}},acc{{1}}){{{}}}'.format(combos.name.index(c))
        u[name] = lambda: c@near1()

In [53]:
def _add_minmax_pools(ww, h, lag, *, Z, z): #.-`~,-'~.-`~,-'~.-`~,-'~.-`~,-'~ HELPER FUNCTION!
    strh = str(h)
    T = OX()
    
    guy = lambda: Z[ww][-h:]
    if lag == 0: # includes the case when we have 'volatility'
        T[f'{ww}Since{strh}*'] = lambda: guy(); since = lambda: T[f'{ww}Since{strh}*']
    else:
        T[f'{ww}Since{strh}*'] = lambda: np.nancumsum(guy(), axis=0); since = lambda: T[f'{ww}Since{strh}*']
        z[f'{ww}Since{strh}'] = lambda: since()[-1]
        
    
    for mm, MM, flag, updown, nanfunc in [('max','Max',-np.inf,'down',np.nanmin), ('min','Min',np.inf,'up',np.nanmax)]:
        def _mmSince():
            sinceForMM = since().copy()
            sinceForMM[np.isnan(since())] = flag
            mmSince = (np.maximum if mm=='max' else np.minimum).accumulate(sinceForMM, axis=0)
            mmSince[mmSince==flag] = np.nan
            return mmSince
        T[f'{ww}{MM}Since{strh}*'] = _mmSince; mmSince = lambda: T[f'{ww}{MM}Since{strh}*']
        z[f'{ww}{MM}Since{strh}'] = lambda: mmSince()[-1]
        
        T[ww+f'Draw{updown}1Since{strh}*'] = lambda:since()-mmSince(); drawSince = lambda:T[ww+f'Draw{updown}1Since{strh}*']
        z[f'{ww}Draw{updown}1Since{strh}'] = lambda: drawSince()[-1]
        
        # if mm=='max' Drawdown is negative so we take np.nanmin; if mm=='min' Drawup is positive so we take np.nanmax;
        z[f'{ww}Draw{updown}5Since{strh}'] = lambda: nanfunc(drawSince()[-5:], axis=0)
        z[f'{ww}Draw{updown}10Since{strh}'] = lambda: nanfunc(drawSince()[-10:], axis=0)
        z[f'{ww}Draw{updown}20Since{strh}'] = lambda: nanfunc(drawSince()[-20:], axis=0)
        z[f'{ww}Draw{updown}(10-5)Since{strh}'] = lambda: nanfunc(drawSince()[-10:-5], axis=0)
        z[f'{ww}Draw{updown}(20-10)Since{strh}'] = lambda: nanfunc(drawSince()[-20:-10], axis=0)
        z[f'{ww}{MM}Since{strh}{{5}}'] = lambda: mmSince()[-6]
        z[f'{ww}{MM}Since{strh}{{10}}'] = lambda: mmSince()[-11]
        z[f'{ww}{MM}Since{strh}{{20}}'] = lambda: mmSince()[-21]

In [54]:
def add_longterm_price_features(*,U,u,horizons=[21,62,125,250]):
    # longterm return horizons ################################################################ oo + aoo already in mm_pool
#     for ww in ['oo','aoo','(oo-aoo)']:
#         U[ww+'60'] = lambda: np.nansum(_rollwind(U[ww+'20'],3,20), axis=-1), lambda: np.nansum(U[ww+'20'][-41::20], axis=0)
#         u[ww+'120'] = lambda: np.nansum(U[ww+'60'][-61::60], axis=0)
    # MINMAX POOLS ################################################################### no NaNs
    for h in horizons:
        ##### historical liquity #NEW
        u['fracRecordedDaysSince'+str(h)] = lambda: (~np.isnan(U.volume[-h:])).sum(axis=0) / h
        ##### drawdowns and drawups
        for ww, lag in [('oo1',1),('aoo1',1),('(oo-aoo)1',1), ('oo10',10),('aoo10',10),('(oo-aoo)10',10),
                        ('vp1dd1',1),('vp5dd1',1),('vp10dd1',1), ('volatility10',0),('volatility20',0)]:
            _add_minmax_pools(ww, h, Z=U, z=u, lag=lag)

In [55]:
def add_global_price_features(*,U,u,V,v,horizons=[21,62,125,250]):
    ''''m_' prefix means "market", i.e. averaged (weights uniform) across the market at each time
    'wVPm_' prefix means "market, weighted by volume*price"'''
#     V['m_vp1dd1'] = (lambda: noinf_log(V.m_vp1) - noinf_log(np.shift(V.m_vp1,1,axis=0)),
#                      lambda: noinf_log(V.m_vp1[-1]) - noinf_log(V.m_vp1[-2]))
    ################################################################## market returns, no NaNs #NEW
    V['m_vp5_'] = lambda: noinf_log( np.nansum(U.vp5, axis=1) ), lambda: noinf_log( np.nansum(U.vp5[-1]) )
    V['m_vp5dd1'] = lambda: V.m_vp5_ - np.shift(V.m_vp5_,1,axis=0), lambda: V.m_vp5_[-1] - V.m_vp5_[-2]
    V['m_oo1'] = (lambda: noinf_log( np.nanmean( np.exp(U.oo1) , axis=1) ),
                  lambda: noinf_log( np.nanmean( np.exp(U.oo1[-1]) ) ))
    V['wVPm_oo1'] = (lambda: noinf_log( wmean( np.exp(U.oo1) , np.shift(U.vp1,1,axis=0), axis=1) ),
                     lambda: noinf_log( wmean( np.exp(U.oo1[-1]) , U.vp1[-2]) ))
    for i in [5,10,20,60]:
        s = str(i) 
        # this 'm_oo{i}' is first taking returns over horizon for each stock, then averaging uniformly across market
        if i < 20:
            v['m_oo'+s] = lambda: noinf_log( np.nanmean( np.exp(U['oo'+s][-1]) ) )
        else:
            v['m_oo'+s] = lambda: noinf_log( np.nanmean( np.exp(u['oo'+s]) ) )
        # we are doing log change of average 5 day volume*price speard globally
        v['m_vp5dd'+s] = lambda: V.m_vp5_[-1] - V.m_vp5_[-1-i]
        # 'wVPm_' returns are first averaged horizontally across market, then averaged vertically across time using
        # changing market volume*price weight, and this is somewhat necessary because volume*price changes across time
        v['wVPm_oo'+s] = lambda: np.nansum(V.wVPm_oo1[-i:])
        # volatility here. just, straightforward calc off the global returns
        v['m_volatility'+s] = lambda: np.nanstd(V.m_oo1[-i:])
        v['wVPm_volatility'+s] = lambda: np.nanstd(V.wVPm_oo1[-i:])
    ########################################################################
#     for h in horizons:
#         strh = str(h)
#         for ww, lag in [('m_vp1dd1',1),('m_vp5dd1',1),('m_oo1',1),('m_oo10',10),('wVPm_oo1',1),('wVPm_oo10',10),
#                         ('m_volatility10',0),('m_volatility20',0),('wVPm_volatility10',0),('wVPm_volatility20',0)]:
#             _add_minmax_pools(ww, h, Z=V, z=v, lag=lag)

In [56]:
def add_postprocessing_features(*,F):
    ################## systematic assetCode encodings ####################
    F['assetCode_inUniCount'] = F.assetCodeId.map(inUniCountSeries)
    
    ############# random assetCode encodings ################################
    for i in range(1,11):
        F['assetCode_randMap32_'+str(i**2)] = F.assetCodeId.map(randMap32(i**2)).astype(float)
    #F['assetCodeIdWTFlogabs'] = F.assetCodeId.map(str).map(hash).pipe(np.abs).pipe(np.log)
    #F['assetCodeIdWTFsin2'] = F.assetCodeId.map(str).map(hash).pipe(np.sin).pipe(lambda x: x * x)
    F['assetCodeIdWTFFlogabs'] = F.assetCodeId.map(dfHack2.assetCodeIdWTFFlogabs)
    F['assetCodeIdWTFFsin2'] = F.assetCodeId.map(dfHack2.assetCodeIdWTFFsin2)

### Some rather unfortunate hack-like code to reliably generate features day-by-day follows until the next section

Honestly don't bother trying to understand the code from here up till the header "**HELLO READ THIS**"

It's all just plumbing work to get the feature generation code above to work in production.

In [57]:
def make_single_A_U_u_V_v(*,M):
    '''M needs to assetCodeId indexed'''
    assert M.index.name == 'assetCodeId', "argument to make_single_A_U_u_V_v must be assetCodeId-indexed"
    A = pd.DataFrame(index=stocksInUni).join(M, how='left') #EDITLINE
    #^ A = join onto standard assetCode row-index
    U = OX(**{c:A[c].values[np.newaxis,:] for c in columns_for_U},
          **{c+'1':np.nan_to_num(A[c].values[np.newaxis,:]) for c in set(returns_columns.values())})
    if OX.record_dependency:
        for c in (c for c in U if c not in OX.depend_graph):
            U[c] = U[c] # for building dependency graph
    return A, U, OX(), OX(), OX()

def make_U(*,M):
    Us = [make_single_A_U_u_V_v(M=m.set_index('assetCodeId'))[1] for _,m in M.groupby('time')]
    U = OX(**{c:np.concatenate([U[c] for U in Us],axis=0) for c in Us[0]})
    return U

In [58]:
# do a minimal initial run of feature generation to get the static list of columns of interest and
# also generate feature dependency and the set of features to allow
OX.allow = lambda x: x in themodel.xAllowed

Minit = M[M.time<=M.time.unique()[longterm]]
set_basic_features(M=Minit)
Uinit = make_U(M=Minit)

OX.record_dependency = True; OX.depend_graph = {}

uinit = OX()
BUG.U0_keys = set(dict.keys(Uinit))
add_shortterm_price_features(U=Uinit,u=uinit)
add_longterm_price_features(U=Uinit,u=uinit)
Vinit, vinit = OX(), OX()
add_global_price_features(U=Uinit,u=uinit,V=Vinit,v=vinit)

OX.record_dependency = False; OX.build_dependency()

Mcolumns = [c for c in Minit.columns if c not in excluded_columns]
xMcolumns = set(Mcolumns)
Ucolumns = [c for c in Uinit if c not in Mcolumns and c[-1] not in '*_' and c in OX.depends]
ucolumns = [c for c in uinit if c not in Mcolumns and c[-1] not in '*_' and c in OX.depends]
Vcolumns = [c for c in Vinit if c not in Mcolumns and c[-1] not in '*_' and c in OX.depends]
vcolumns = [c for c in vinit if c not in Mcolumns and c[-1] not in '*_' and c in OX.depends]
xUcolumns,xucolumns,xVcolumns,xvcolumns = [set(C) for C in [Ucolumns,ucolumns,Vcolumns,vcolumns]]
for i, a in enumerate([xUcolumns,xucolumns,xVcolumns,xvcolumns]):
    for j, b in enumerate([xUcolumns,xucolumns,xVcolumns,xvcolumns]):
        assert a is b or len(a&b)==0, f'a and b: {i} {j}'

del Minit,Uinit,uinit,Vinit,vinit; gc.collect()

119

In [59]:
def iter_features_templates(MNP_iter, *, U_, V_, train=False):
    U_, termU = U_
    V_, termV = V_
    for M, N, P in MNP_iter:
        orig_index = M.index
        set_basic_features(M=M) # adds "long" features and deletes non-log price features
        M.set_index('assetCodeId', inplace=True)
        A, U, u, V, v = make_single_A_U_u_V_v(M=M); del V
        
        assert BUG.U0_keys==set(dict.keys(U))
        M.drop(columns=[c for c in M.columns if c not in xMcolumns], inplace=True)
        
        try:
            akey = next(iter(U))
            assert U[akey].shape[1] == U_[akey].shape[1]
            # expand rows in old U_ if new assetCodeIds were seen #NOTE don't need this anymore, decided to only use inUni assets
    #         akey = next(iter(U))
    #         diff = U[akey].shape[1] - U_[akey].shape[1]
    #         if diff > 0:
    #             for key in U_:
    #                 if key[-1] != '_': # normal features
    #                     U_[key] = np.pad(U_[key], pad_width=((0,0),(0,diff)), mode='constant', constant_values=np.nan)
    #                 else: # NaN-filled-with-0 features
    #                     U_[key] = np.pad(U_[key], pad_width=((0,0),(0,diff)), mode='constant', constant_values=0)
        except StopIteration: # no features in U
            pass
        
        # join new row vectors in U to the old tables in U_
        for key in U:
            U_[key] = np.concatenate([U_[key],U[key]], axis=0)
        xsingleUcolumns = set(U); del U
        
        # retain dimensions of U_ and V_ objects for sanity check after feature generation
        _U_shape, _V_shape = {k:x.shape for k,x in dict.items(U_)}, {k:x.shape for k,x in dict.items(V_)}
        
        # add features to U_ and u and V and v
        gc.collect()
        add_shortterm_price_features(U=U_,u=u)
        gc.collect()
        add_longterm_price_features(U=U_,u=u)
        gc.collect()
        add_global_price_features(U=U_,u=u,V=V_,v=v)
        gc.collect()
        assert len(set(U_)&set(u))==0 and len(set(V_)&set(v))==0
        
        # run time check that the shape of the U things are ok
        ##############################################BEGIN####CHECK####################################################
        #dict.update(BUG, U=U,U_=U_,V_=V_,u=u,v=v,_U_shape=_U_shape,_V_shape=_V_shape)
#         assert all(c in xMcolumns or c[-1] in '*_' for c in set(U_)^xUcolumns), (
#             '!F! columns found in U_ not in Ucolumns must be in Mcolumns or end in exclusion suffixes')
        #
        try: # for U, u
            c0 = next(iter(Ucolumns))
            for c in Ucolumns:
                assert len(U_[c].shape)==2 and U_[c].shape[1]==U_[c0].shape[1], '!F! U_[c] must have expected shape'
                if c in xsingleUcolumns:
                    assert U_[c].shape==_U_shape[c], '!F! U_[c] shape must be consistent with same from make_single_A_U_...'
                else:
                    assert U_[c].shape[0]==_U_shape[c][0]+1 and U_[c].shape[1]==_U_shape[c][1], '!F! U_[c] shape consistency'
            assert all(c in xMcolumns or c[-1] in '*_' for c in set(u)^xucolumns), (
                '!F! columns found in u not in ucolumns must be in Mcolumns or end in exclusion suffixes')
            for c in ucolumns:
                assert len(u[c].shape)==1 and u[c].shape[0]==U_[c0].shape[1], (
                    f'`u` items shape must match last dimension: c={c}, u={u[c].shape}, U_={U_[c0].shape}')
        except StopIteration: # U has no features (#TODO also doesn't check u even if u has features)
            pass
        #                for V,v
        try: 
            assert all(c in xMcolumns or c[-1] in '*_' for c in set(V_)^xVcolumns), (
                '!F! columns found in V_ not in Vcolumns must be in Mcolumns or end in exclusion suffixes')
            c0 = next(iter(Vcolumns))
            for c in Vcolumns:
                assert len(V_[c].shape)==1 and V_[c].shape[0]==V_[c0].shape[0], 'V_[c] shape must be consistent'
                assert V_[c].shape[0]==_V_shape[c][0]+1, 'V_[c].shape[0] must grow by 1 each day'
            assert all(c in xMcolumns or c[-1] in '*_' for c in set(v)^xvcolumns), (
                '!F! columns found in v not in vcolumns must be in Mcolumns or end in exclusion suffixes')
        except StopIteration: # V has no features
            pass
        for c in vcolumns:
            assert v[c].shape==(), 'v[c] must be scalar (i.e. shape=()) shaped'
        #################################################END#######CHECK###############################################
        
        # cutoff excess past terms to save memory
        for key in U_:
            if len(U_[key])>termU:
                U_[key] = U_[key][-termU:]
        for key in V_:
            if len(V_[key])>termV:
                V_[key] = V_[key][-termV:]
        
        # take out the bottom row of the feature tables to feed out as our feature construction iterator
        _wide = np.zeros(u[next(iter(u))].shape, dtype=np.int8) # dtype as np.int8 to make it not upcast orig dtype
        _dict = {c: U_[c][-1] for c in Ucolumns}
        _dict.update({c: u[c] for c in ucolumns})
        _dict.update({c: V_[c][-1] + _wide for c in Vcolumns})
        _dict.update({c: v[c] + _wide for c in vcolumns})
        del u, v
#         dict.update(BUG, U_=U_,V_=V_,u=u,v=v)
        # piece everything together
        columns = Ucolumns + ucolumns + Vcolumns + vcolumns
        F = pd.DataFrame([_dict[c] for c in columns], index=columns, columns=A.index).T
        F = M.join(F, how='left')
        assert 'assetCodeId' not in F.columns
        F.reset_index(inplace=True)
        F.index = orig_index
        
        if train: # remove rows of stocks that are not inUni
            keep = F.assetCodeId.isin(xStocksInUni).values & (P.universe!=0).values
            F, P = F[keep], P[keep]
        
#         for c in enumeration_columns:
#             assert np.issubdtype(F[c].dtype, np.integer)
        add_postprocessing_features(F=F)
        
        # downgrade columns to 32bit dtypes for memory saving
#        convert32(F) #FLOAT32 # we don't do this anymore because memsave_concat can do it for us
        yield F, P
        del F, P; gc.collect()

In [60]:
# make initial historical U_ and u_ for creating the whole training features #EDITNOTE

def iter_M_N_P_train(*,M,N):
    assert M.time.iloc[0] >= train_start_time, '`iter_M_N_P_train` needs to be fed `M` and `N` filtered on the training days'
    M_, N_ = M, N
    for time, M in M_.groupby('time'):
#         if time<train_start_time: # no longer needed because I enforce feeding M already filtered for the training days
#             continue
        #M['quarter'] = M.time.dt.year+(M.time.dt.quarter-1)/4
        P = M[[c for c in excluded_columns if c in M]] #EDITLINE
        M = M.drop(columns=['returnsOpenNextMktres10'])#,'quarter'])
        yield M, None, P
        
Mstart = M[M.time<train_start_time]
set_basic_features(M=Mstart)
U_ = make_U(M=Mstart)
u_ = OX()
add_shortterm_price_features(U=U_,u=u_)
add_longterm_price_features(U=U_,u=u_)
V_, v_ = OX(), OX()
add_global_price_features(U=U_,u=u_,V=V_,v=v_)
del Mstart, u_, v_; gc.collect()

Mtrain = M[M.time>=train_start_time]
#del M; gc.collect() #NOPE #TODO? this is totally wrong I need M after to generate the test U_ and V_

##### make train helper

In [77]:
def memsave_concat(train):
    m = (Mtrain.assetCode.map(assetCodeIdAssign).isin(xStocksInUni) & (Mtrain.universe!=0)).sum()
    P_ = []
    for k, (F, P) in enumerate(train):
        if k == 0:
            col = F.columns
            ans = np.zeros([m,len(col)], dtype=np.float32) # #FLOAT32
            m0 = 0
        assert (F.index == P.index).all()
        P_.append(P)
        del P
        m1 = m0 + len(F)
        assert (F.columns == col).all()
        ans[m0:m1] = F
        del F; gc.collect()
        m0 = m1
    Pret = pd.concat(P_, axis=0)
    Fret = pd.DataFrame(ans, index=Pret.index, columns=col, copy=False)
    return Fret, Pret

#### The cell to generate the training features. WARNING: takes a long time!

In [199]:
%%time
def make_train():
    global U_, V_
    feat_iter = iter_features_templates(iter_M_N_P_train(M=Mtrain,N=None), U_=(U_,longterm), V_=(V_,longterm), train=True)
    train = ( (print_progress(i,10), FP)[-1] for i,FP in enumerate(feat_iter) )
    del U_, V_; gc.collect()
    F, P = memsave_concat(train)
    F.index = P.index
    return F,P
F,P = make_train()

0.........100.........200.........300.........400.........500.........600.........700.........800.........900.........1000.........1100.........1200.........1300.........1400.........1500.........1600.........1700.........1800.........1900.........2000.CPU times: user 2h 1min 53s, sys: 2min 29s, total: 2h 4min 22s
Wall time: 1h 37min 3s


In [ ]:
W, D = make_W_D()

#### saving some variables for leaderboard submission loop before deletion

In [61]:
# defines U_ ahead of time for initial start of the feature processing loop in submission stage

Mstart = M[M.time>=M.time.unique()[-longterm]]
set_basic_features(M=Mstart)
U_ = make_U(M=Mstart)
del Mstart; gc.collect()
u_, V_, v_ = OX(), OX(), OX()
add_shortterm_price_features(U=U_,u=u_)
add_longterm_price_features(U=U_,u=u_)
add_global_price_features(U=U_,u=u_,V=V_,v=v_)
del u_, v_; gc.collect()

0

In [62]:
#del M; gc.collect() # We still need U_ and V_ for final submission loop

# HELLO READ THIS

By now we have DataFrames F and P
- F = The features data. shape = ({training and cv days}, {all the features})
- P = The target and other columns not in F. Same index as F

import to note,
- columns under F: assetCodeId
- columns under P: assetCode, time, quarter

##### some P columns setup

In [ ]:
##P.rename(columns={'returnsOpenNextMktres10':'y'}, inplace=True)
P['y'] = P.returnsOpenNextMktres10
#globalStd = (P.universe*P.y).std()
#grp = P.groupby(F.assetCodeId)[['y','universe']]
#assetStds = grp.apply(lambda x: np.sqrt((x.universe*x.y**2).sum()/(x.universe).sum()) if x.universe.sum()>25 else globalStd)
#P['assetCodeId'] = P.assetCode.map(assetCodeIdAssign)
#P.y /= P.assetCodeId.map(assetStds)

In [203]:
setup_P(F=F,P=P,vp=False)

#### _~ ~ ~ ~ ~ saving/loading for local testing_ #TODO #KILL

In [80]:
# pd.to_pickle((F,P),'../data/saves/ikg.test.pkl')

In [64]:
#pd.to_pickle((F,P), '/big/data/saves/train_5fixedsince.32.pkl')
#pd.to_pickle((F,P), '/big/data/saves/train_5fixedsince+1threecombos.32.pkl')
F,P = pd.read_pickle('../data/saves/train_5fixedsince+1.32.pkl')
P.rename(columns={'weight':'flat_weight'}, inplace=True)
W, C, D = pd.read_pickle('../data/pair/working_2ONEorTEN.pkl'); W, C, D = O(**W), O(**C), O(**D); del C; gc.collect()

7

In [96]:
#pd.to_pickle((F,P), '/big/data/saves/train_4Newnewfault64xx.pkl')
#F,P = pd.read_pickle('/big/data/saves/train_4Newnewfault64xx.pkl')

In [51]:
# FF,PP = pd.read_pickle('/big/data/saves/train_3allright32.pkl')

In [52]:
#F0,P0 = pd.read_pickle('/big/data/saves/train_thefault.pkl')

In [53]:
# F,P = FF,PP

## @#@///////////////////////////////////// Training some particular model! /////////////////////////////////@#@

In [65]:
def lgb_kaggle_metric(preds, valid_data):
    df_time = valid_data.timeFactor
    #labels = valid_data.get_label()
    values = valid_data.value
    #assert len(labels) == len(df_time)

    preds = preds*2-1
    #labels = labels*2-1
    x_t = preds * values
    
    # Here we take advantage of the fact that `labels` (used to calculate `x_t`)
    # is a pd.Series and call `group_by`
    x_t_sum = x_t.groupby(df_time).sum()
    score = x_t_sum.mean() / x_t_sum.std()

    valid_data.i += lgb_kaggle_metric.hack
    return 'kaggle', score+valid_data.i, True
lgb_kaggle_metric.hack = 0

### load the lgb data structure and train and evaluate

In [66]:
def create_booster(model, *, F, P):
    F_, P_ = F[model.features], P.copy() #TODO #MEM
    lgb_data_info = dict(
        feature_name = list(F_.columns),
        categorical_feature = list(F_.dtypes[F_.dtypes.isin([np.int64,np.int32])].index),
        free_raw_data = True,
    )
    theweight = model.weight
    ho = P_.quarter >= 2015.5
    tr = P_.quarter.isin(model.train_on)
    #cv = (P_.time>=train_start_time) & ~(tr|ho)
    L = O()
    L.tr = lgb.Dataset(F_[tr], P_.target[tr], weight=P_[theweight][tr], **lgb_data_info)
    #L.cv = lgb.Dataset(F_[cv], P_.target[cv], reference=L.tr, weight=P_[theweight][cv], **lgb_data_info)
    L.ho = lgb.Dataset(F_[ho], P_.target[ho], weight=P_[theweight][ho], **lgb_data_info)
    
    L.tr.timeFactor = P_.time[tr].factorize()[0]
    #L.cv.timeFactor = P_.time[cv].factorize()[0]
    _value = (P_.upDown*P_[theweight])
    L.tr.value = _value[tr]
    #L.cv.value = _value[cv]
    L.tr.i = 0
    #L.cv.i = 0
    
    params = dict(
        objective = 'binary',
        #num_iterations = 450,
        #early_stopping_round = 32,
        metric = 'None',
        seed = 44,
        bagging_seed = 45,
        feature_fraction_seed = 46,
    )
    params.update(model.params)
    global VERBOSE_EVAL
    model.bst = bst = lgb.train(params, L.tr, valid_sets=[L.tr], valid_names=['tr'],
                                feval=lgb_kaggle_metric, verbose_eval=VERBOSE_EVAL)
    if VERBOSE_EVAL:
        print('='*60)
    P_['guess'] = bst.predict(F_)*2-1
    P_['trade'] = P_.guess*P_.upDown*P_[theweight]
    daily = P_[~ho].groupby('time').trade.sum()
    bst.std = daily.std(ddof=0)

In [103]:
def make_pair_F_P(model, *, F, P):
    if not hasattr(model, 'filts'):
        model.x_copy_features = set(model.copy_features)
        model.x_diff_features = set(model.diff_features)
        model.x_sign_features = set(model.sign_features)
        model.x_corr_features = set(model.corr_features)
        filts = model.filts = [lambda x,s=s,model=model: x in getattr(model, f'x_{s}_features')
                               for s in 'copy diff sign corr'.split()]
    else:
        filts = model.filts

    feats = list(filter(lambda x: filts[0](x)|filts[1](x)|filts[2](x), F.columns))
    if 'assetCodeId' not in feats:
        feats = ['assetCodeId'] + feats
        
    CC, G, Q = make_CC_G_Q_from_Dl_F_P_W(
        F=F, P=P, W=W, Dl=D.av10,
        lo = model.Pair.lo,
        hi = model.Pair.hi,
        repeat = model.Pair.repeat
    )

    FG, PQ = make_FG_PQ_from_CC_G_Q(*filts, CC=CC, G=G, Q=Q)

    model.features = list(FG.columns)
    return FG, PQ

In [142]:
def resolve_pair_guess(model, guess, P, PQ):
    P['assetCodeId'] = P.assetCode.map(assetCodeIdAssign)
    PQ['__0__assetCodeId'] = PQ.__0__assetCode.map(assetCodeIdAssign)
    PQ['__1__assetCodeId'] = PQ.__1__assetCode.map(assetCodeIdAssign)
    PQ['guess'] = guess * PQ[model.weight]
    
    dummy = P.set_index(['time','assetCodeId'])[[]]
    PQ.rename(columns={'__0__assetCodeId':'assetCodeId'}, inplace=True)
    buy = dummy.join(PQ.groupby(['time','assetCodeId']).guess.sum()).fillna(0)
    PQ.rename(columns={'assetCodeId':'__0__assetCodeId','__1__assetCodeId':'assetCodeId'}, inplace=True)
    sell = dummy.join(PQ.groupby(['time','assetCodeId']).guess.sum()).fillna(0)
    PQ.rename(columns={'assetCodeId':'__1__assetCodeId'}, inplace=True)
    
    return (buy-sell).values

### DO IT. Train the models.

In [75]:
%%time
VERBOSE_EVAL = 50
for model, c in zip(themodel.solo.specs, themodel.solo.coefs):
    if c != 0:
        create_booster(model,F=F,P=P)

[50]	tr's kaggle: 0.894557
[100]	tr's kaggle: 1.02789
[150]	tr's kaggle: 1.13252
[200]	tr's kaggle: 1.21308
[250]	tr's kaggle: 1.27962
[300]	tr's kaggle: 1.3391
[350]	tr's kaggle: 1.39301
[400]	tr's kaggle: 1.44017
[450]	tr's kaggle: 1.47991
CPU times: user 44min 35s, sys: 18.1 s, total: 44min 53s
Wall time: 5min 44s


In [107]:
%%time
VERBOSE_EVAL = 50
SEE = print
for model, c in zip(themodel.pair.specs, themodel.pair.coefs):
    if c != 0:
        FG, PQ = make_pair_F_P(model,F=F,P=P)
        create_booster(model,F=FG,P=PQ)
SEE = lambda*a,**k:None

(pair) len(F.columns) = 1259
(pair) generating [350]0.........100.........200.........300....
(pair) putting together... (pair) done
[50]	tr's kaggle: 1.0072
[100]	tr's kaggle: 1.44753
[150]	tr's kaggle: 1.77931
[200]	tr's kaggle: 1.976
[250]	tr's kaggle: 2.09887
CPU times: user 28min 14s, sys: 25.2 s, total: 28min 40s
Wall time: 5min 2s


In [150]:
TRAIN = False

### define the model aggregating function

In [160]:
@inside(themodel)
def guess(*, F, P):
    weight_map_multiplier = {'vp1_weight':F.vp1/1e9,'vp10_weight':F.vp10/1e9,'flat_weight':1,}
    
    def _pair_func(model, *, F, P):
        FG, PQ = make_pair_F_P(model,F=F,P=P)
        guess = model.bst.predict(FG) * 2 - 1
        del FG, PQ; gc.collect()
        return resolve_pair_guess(model, guess, P=P,PQ=PQ)
    pairs = [np.nan_to_num(_pair_func(s,F=F,P=P)) * c / s.bst.std
             for s,c in zip(themodel.pair.specs,themodel.pair.coefs) if c!=0]
    
    solos = [np.nan_to_num((s.bst.predict(F[s.features])*2-1) * weight_map_multiplier[s.weight]) * c / s.bst.std
             for s,c in zip(themodel.solo.specs,themodel.solo.coefs) if c!=0]
    
    return (sum(solos+pairs)/1000000).clip(-1,1)

### last look at the model blending

In [109]:
[print((spec.__name__ if spec is not None else 'None'), end=', ') for spec in themodel.solo.specs];

IKGCover, 

In [54]:
# themodel.solo.coefs = []

In [110]:
[print((spec.__name__ if spec is not None else 'None'), end=', ') for spec in themodel.pair.specs];

pIKGCoverHomo, 

In [111]:
# themodel.pair.coefs = []

### Look at the holdout results

In [161]:
ho = P.time == P.time.iloc[-1]
SEE = print
def view_results(ho, *, F, P):
    F, P = F[ho], P[ho]
    P['guess'] = themodel.guess(F=F,P=P)
    P.guess = P.guess*(np.abs(P.guess)>=0.0)

    P['trade'] = P.guess*P.upDown*P.flat_weight

    daily = P[ho].groupby('time').trade.sum()
    print(daily.mean()/daily.std(ddof=0))
    plt.hist(daily, bins=80);
    
    time = P[ho].groupby('time').time.first()
    market = F[ho].groupby(P.time).m_oo1.first()
    trade = P[ho].groupby('time').trade.sum()
    
    %matplotlib inline
    canvas(12,6)
    plt.plot(time, market/market.std(ddof=0), linewidth=.2);
    plt.plot(time, trade/trade.std(ddof=0), linewidth=1);

view_results(ho, F=F,P=P)

(pair) len(F.columns) = 1259
(pair) generating [350]0

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2014 but corresponding boolean dimension is 1

In [159]:
BUG.P

,time,assetCode,assetName,universe,returnsOpenNextMktres10,quarter,y,target,upDown,absVal,flat_weight,vp1_weight,vp5_weight,vp10_weight,always_weight,assetCodeId
732812,2009-01-02 22:00:00+00:00,A.N,Agilent Technologies Inc,1.0,0.179633,2009.00,0.179633,True,1,0.179633,0.179633,0.008491,0.011776,0.015845,0.179633,0
732814,2009-01-02 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1.0,0.029782,2009.00,0.029782,True,1,0.029782,0.029782,0.000803,0.000745,0.001058,0.000000,2
732815,2009-01-02 22:00:00+00:00,AAPL.O,Apple Inc,1.0,-0.026166,2009.00,-0.026166,False,-1,0.026166,0.026166,0.060380,0.053807,0.055439,0.026166,3
732816,2009-01-02 22:00:00+00:00,AB.N,AllianceBernstein Holding LP,1.0,0.046983,2009.00,0.046983,True,1,0.046983,0.046983,0.000652,0.000559,0.000546,0.000000,1564
732817,2009-01-02 22:00:00+00:00,ABB.N,ABB Ltd,1.0,-0.009046,2009.00,-0.009046,False,-1,0.009046,0.009046,0.000455,0.000386,0.000484,0.009046,4
732818,2009-01-02 22:00:00+00:00,ABC.N,AmerisourceBergen Corp,1.0,0.057848,2009.00,0.057848,True,1,0.057848,0.057848,0.004098,0.002195,0.002336,0.057848,5
732821,2009-01-02 22:00:00+00:00,ABT.N,Abbott Laboratories,1.0,-0.026621,2009.00,-0.026621,False,-1,0.026621,0.026621,0.009000,0.006653,0.008801,0.026621,8
732822,2009-01-02 22:00:00+00:00,ABV.N,Companhia de Bebidas das Americas Ambev,1.0,0.039785,2009.00,0.039785,True,1,0.039785,0.039785,0.000898,0.000927,0.001215,0.000000,9
732824,2009-01-02 22:00:00+00:00,ACC.N,American Campus Communities Inc,1.0,0.047610,2009.00,0.047610,True,1,0.047610,0.047610,0.000747,0.000604,0.000850,0.000000,1948
732825,2009-01-02 22:00:00+00:00,ACE.N,Chubb Ltd,1.0,-0.046250,2009.00,-0.046250,False,-1,0.046250,0.046250,0.002150,0.002674,0.003556,0.000000,2087


## +-+-+-+-+-+-+-+-+-+-+-+-+-+-+- Prediction and submission +-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-

#### the rest of the administrative pipeline

In [59]:
__env__ = _make_env() #TODO #KILL

In [60]:
%%time
def do_submission_loop():
    feat_iter = iter_features_templates(__env__.get_prediction_days(), U_=(U_,longterm), V_=(V_,longterm))
    for i, (f, p) in enumerate(feat_iter):
        print_progress(i)
        p_ = p.reset_index()
        p_['time'] = 0
        p_['pair_weight']
        p.confidenceValue = ans = themodel.guess(F=f,P=p_)
        if np.isnan(ans).any() or (ans<-1).any() or (ans>1).any():
            BUG.i, BUG.f, BUG.p, BUG.ans = i, f, p, ans
            assert False
        __env__.predict(p)
        gc.collect()
do_submission_loop()

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........500.........510.........520.........530.........540.........550.........560.........570.........580.........590.........600.........610.........620.........630........CPU times: user 57min 32s, sys: 2min, total: 59min 33s
Wall time: 18min 38s


In [61]:
__env__.write_submission_file()

Your submission file has been saved. Once you `Commit` your Kernel and it finishes running, you can submit the file to the competition from the Kernel Viewer `Output` tab.


# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOCAL TESTING ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 